<a href="https://colab.research.google.com/github/Jc7796/pymillion/blob/main/Script_Loto_Analyse_V3_(Quasi_Gagnants_D%C3%A9taill%C3%A9s_Fix_NumPy).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:


import os
import logging
from io import StringIO
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
# from bs4 import BeautifulSoup # Suppression de l'import de BeautifulSoup
from scipy.stats import chisquare, chi2_contingency
from collections import Counter, defaultdict # defaultdict ajouté
from itertools import combinations, product
import time
from math import comb as nCr
import json
import random

# --- Machine Learning Imports ---
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import statsmodels.api as sm # Pour l'analyse de saisonnalité/cycles (ACF/PACF)
from statsmodels.tsa.arima.model import ARIMA # Pour les modèles AR, MA, ARMA, ARIMA
from statsmodels.stats.diagnostic import acorr_ljungbox # Pour le test de Ljung-Box sur les résidus
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
from prefixspan import PrefixSpan
import pmdarima as pm # Ajout de l'import pour auto_arima

# Configuration logs
logging.basicConfig(level=logging.INFO, filename='loto_statistical_analysis.log', filemode='a',
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Dictionnaire pour la conversion des mois français (mis à jour)
MOIS_MAP = {
    'janvier': '01', 'février': '02', 'fevrier': '02', 'mars': '03',
    'avril': '04', 'mai': '05', 'juin': '06', 'juillet': '07',
    'août': '08', 'aout': '08', 'septembre': '09', 'octobre': '10',
    'novembre': '11', 'décembre': '12', 'decembre': '12'
}

# --- Paramètres Globaux ---
USE_GRID_SEARCH_FOR_FULL_ANALYSIS_ML = True
MAX_GRID_SEARCH_CUSTOM_COMBINATIONS = 133
RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS = True
RUN_BACKTESTING_ML_IN_FULL_ANALYSIS = True
RUN_DEEP_DIVE_COMPLEMENTARY_9 = True
RUN_DEEP_DIVE_CLUSTER_5 = True
RUN_TRANSITION_PROBABILITIES = True
RUN_SLIDING_WINDOW_ENHANCED_METRICS = True
RUN_SLIDING_WINDOW_TIMESERIES_ANALYSIS = True
RUN_DRAW_CLUSTERING = True
RUN_ALMOST_WINNERS_ANALYSIS_IN_BACKTEST = True # S'assurer que c'est True
RUN_PREFIXSPAN_ANALYSIS = True

# --- Drapeau pour choisir le mode d'exécution ---
RUN_LAMBDA_STRATEGY_MODE = False

# --- Vos Numéros Joués ---
MES_NUMEROS_PRINCIPAUX_JOUES = {6, 7, 9, 30}
MES_COMPLEMENTAIRES_POSSIBLES_JOUES = {6, 7, 9}

# --- Configuration Loto pour le mode lambda ---
CONFIG_LOTO_LAMBDA = {
    "total_numbers": 49, # Max number for main draw
    "numbers_to_choose": 5,
    "total_lucky_numbers": 10, # Max number for complementary
    "lucky_numbers_to_choose": 1,
    "min_val_number": 1,
    "min_val_lucky": 1
}

# --- Fonctions de base ---
def convert_french_date(date_text):
    if not isinstance(date_text, str):
        return pd.NaT
    date_text_cleaned = date_text.strip()
    parts = date_text_cleaned.lower().split()
    if len(parts) == 3:
        day, month_fr, year = parts
        month_num = MOIS_MAP.get(month_fr)
        if month_num and year.isdigit() and day.isdigit():
            try:
                day_int = int(day)
                if 1 <= day_int <= 31:
                    formatted_date_str = f"{day.zfill(2)}-{month_num}-{year}"
                    dt_obj = pd.to_datetime(formatted_date_str, format="%d-%m-%Y", errors='coerce')
                    if not pd.NaT is dt_obj: return dt_obj
            except ValueError:
                logging.debug(f"Erreur de conversion du jour en entier pour '{date_text_cleaned}'.")
    dt_obj_fallback = pd.to_datetime(date_text_cleaned, dayfirst=True, errors='coerce')
    if pd.NaT is dt_obj_fallback:
        logging.warning(f"Impossible de convertir la date: '{date_text_cleaned}' (toutes les tentatives ont échoué).")
    return dt_obj_fallback

def fetch_lotto_data(page_url="http://loto.akroweb.fr/loto-historique-tirages/"):
    all_lotto_data_dfs = []
    current_url, pages_fetched_count = page_url, 0
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'}
    expected_page_columns = ['Jour', 'Date', 'N1', 'N2', 'N3', 'N4', 'N5', 'Complementaire']
    while current_url and pages_fetched_count < 1:
        logging.info(f"Récupération des données depuis : {current_url}")
        print(f"Récupération des données depuis : {current_url}")
        pages_fetched_count += 1
        lotto_data_page = pd.DataFrame(columns=expected_page_columns)
        try:
            response = requests.get(current_url, timeout=30, headers=headers)
            response.raise_for_status()
            try:
                raw_dfs_list = pd.read_html(StringIO(response.text), header=None, keep_default_na=False, na_values=[''])
                if raw_dfs_list:
                    raw_df = raw_dfs_list[0]
                    temp_data = []
                    if raw_df.shape[1] >= 10:
                        for idx, row in raw_df.iterrows():
                            try:
                                if len(str(row.iloc[2]).strip()) > 5 and str(row.iloc[4]).strip().isdigit():
                                    jour, date_str = str(row.iloc[1]).strip(), str(row.iloc[2]).strip()
                                    nums_str = [str(row.iloc[i]).strip() for i in range(4, 10)]
                                    if all(s.isdigit() for s in nums_str) and len(nums_str) == 6:
                                        temp_data.append([jour, date_str] + nums_str)
                            except IndexError: logging.warning(f"Ligne {idx} raw_df (interne): IndexError. {row.tolist()}")
                            except Exception as e_row_proc: logging.error(f"Erreur ligne {idx} raw_df (interne): {e_row_proc}. {row.tolist()}")
                        if temp_data: lotto_data_page = pd.DataFrame(temp_data, columns=expected_page_columns)
                        else: logging.warning("Aucune donnée exploitable formatée (raw_df).")
                    else: logging.warning(f"raw_df < 10 colonnes ({raw_df.shape[1]}).")
                else: logging.warning(f"pd.read_html (header=None) n'a retourné aucune table sur {current_url}.")
            except ValueError as ve: logging.warning(f"pd.read_html n'a trouvé aucune table sur {current_url}: {ve}")
            except Exception as e_pd_read: logging.error(f"Erreur pd.read_html sur {current_url}: {e_pd_read}")

            if not lotto_data_page.empty:
                num_cols = ["N1", "N2", "N3", "N4", "N5", "Complementaire"]
                for col in num_cols:
                    if col in lotto_data_page.columns:
                        lotto_data_page[col] = pd.to_numeric(lotto_data_page[col], errors="coerce")
                        if not lotto_data_page[col].isnull().all(): lotto_data_page[col] = lotto_data_page[col].astype('Int64')

                if 'Date' in lotto_data_page.columns:
                    lotto_data_page['Date_avant_conversion'] = lotto_data_page['Date']
                    lotto_data_page['Date'] = lotto_data_page['Date'].apply(convert_french_date)
                    lotto_data_page.dropna(subset=['Date'], inplace=True)

                if num_cols_exist := [c for c in num_cols if c in lotto_data_page.columns]:
                    lotto_data_page.dropna(subset=num_cols_exist, how='any', inplace=True)

                if not lotto_data_page.empty: all_lotto_data_dfs.append(lotto_data_page)
                else: logging.warning(f"Aucune ligne valide après nettoyage pour {current_url}.")
            else: logging.warning(f"lotto_data_page vide pour {current_url} avant nettoyage final.")
            current_url = None
        except requests.exceptions.RequestException as e: logging.error(f"Erreur réseau {current_url}: {e}"); current_url = None
        except Exception as e: logging.error(f"Erreur boucle {current_url}: {e}", exc_info=True); current_url = None

    if not all_lotto_data_dfs: logging.warning("Aucune donnée collectée."); return pd.DataFrame(columns=expected_page_columns)

    final_df = pd.concat(all_lotto_data_dfs, ignore_index=True)
    final_num_cols = ["N1", "N2", "N3", "N4", "N5", "Complementaire"]
    for col in final_num_cols:
        if col in final_df.columns:
            final_df[col] = pd.to_numeric(final_df[col], errors='coerce')
            if not final_df[col].isnull().all(): final_df[col] = final_df[col].astype('Int64')

    if 'Date' in final_df.columns:
        final_df['Date'] = pd.to_datetime(final_df['Date'], errors='coerce')
        final_df.dropna(subset=['Date'], inplace=True)
        final_df.sort_values(by='Date', ascending=False, inplace=True); final_df.reset_index(drop=True, inplace=True)

    if main_num_cols := [c for c in ["N1","N2","N3","N4","N5"] if c in final_df.columns]:
        final_df.dropna(subset=main_num_cols, how='all', inplace=True)

    print(f"Données finales après fetch: {final_df.shape}, Colonnes: {final_df.columns.tolist()}")
    if not final_df.empty: print(final_df.head(3).to_string())
    else: print("DataFrame final vide après fetch.")
    return final_df

# --- Fonctions pour la Stratégie Lambda ---
def calculer_probabilites_lambda(config):
    total_comb_principaux = nCr(config["total_numbers"], config["numbers_to_choose"])
    total_comb_chance = nCr(config["total_lucky_numbers"], config["lucky_numbers_to_choose"])
    proba_rang1 = 1 / (total_comb_principaux * total_comb_chance)
    proba_5_bons_sans_chance = (1 / total_comb_principaux) * \
                               (nCr(config["lucky_numbers_to_choose"], 0) * nCr(config["total_lucky_numbers"] - config["lucky_numbers_to_choose"], config["lucky_numbers_to_choose"]) / total_comb_chance)

    print("--- Probabilités (Exemple Loto France) ---")
    print(f"Nombre total de combinaisons principales : {total_comb_principaux:,}")
    print(f"Nombre total de combinaisons de N° Chance : {total_comb_chance:,}")
    print(f"Nombre total de combinaisons (Rang 1 - 5N + 1C) : {total_comb_principaux * total_comb_chance:,}")
    print(f"Probabilité de gagner le Rang 1 (5N + 1C) : 1 chance sur {1/proba_rang1:,.0f}")
    if proba_5_bons_sans_chance > 0 :
        print(f"Probabilité d'avoir 5 bons numéros (sans N°C) : 1 chance sur {1/proba_5_bons_sans_chance:,.0f} (approximatif)")
    print("-" * 40)

def analyser_donnees_historiques_simplifie_lambda(df_historique, config):
    print("\n[INFO] Lancement des analyses statistiques simplifiées sur l'historique...")
    if df_historique.empty:
        print("[ATTENTION] Historique vide, analyses statistiques non effectuées.")
        return None, None

    num_cols = [f"N{i}" for i in range(1, config["numbers_to_choose"] + 1)]
    frequences_principales_dict, frequences_chance_dict = None, None

    try:
        valid_num_cols = [col for col in num_cols if col in df_historique.columns]
        if valid_num_cols:
            all_numbers_s = df_historique[valid_num_cols].stack()
            all_numbers = pd.to_numeric(all_numbers_s, errors='coerce').dropna().astype(int).values
            if all_numbers.size > 0:
                frequences_principales_dict = Counter(all_numbers)
                print("\n--- Fréquences des Numéros Principaux (Top 5) ---")
                for num, count in frequences_principales_dict.most_common(5): print(f"Numéro {num}: {count} fois")
            else: print("[INFO] Pas de numéros principaux valides trouvés pour l'analyse de fréquence.")
        else: print("[ATTENTION] Colonnes de numéros principaux manquantes dans l'historique.")

        if 'Complementaire' in df_historique.columns:
            lucky_numbers_s = df_historique['Complementaire']
            lucky_numbers = pd.to_numeric(lucky_numbers_s, errors='coerce').dropna().astype(int).values
            if lucky_numbers.size > 0:
                frequences_chance_dict = Counter(lucky_numbers)
                print("\n--- Fréquences des N° Chance (Top 3) ---")
                for num, count in frequences_chance_dict.most_common(3): print(f"N° Chance {num}: {count} fois")
            else: print("[INFO] Pas de numéros chance valides trouvés pour l'analyse de fréquence.")
        else: print("[ATTENTION] Colonne 'Complementaire' manquante dans l'historique.")
    except Exception as e: print(f"[ERREUR] lors de l'analyse simplifiée des fréquences: {e}")

    print("[INFO] Analyses statistiques simplifiées terminées.")
    return frequences_principales_dict, frequences_chance_dict

def generer_combinaison_aleatoire_lambda(config):
    numeros_principaux = sorted(random.sample(range(config["min_val_number"], config["total_numbers"] + 1), config["numbers_to_choose"]))
    numero_chance_list = random.sample(range(config["min_val_lucky"], config["total_lucky_numbers"] + 1), config["lucky_numbers_to_choose"])
    return numeros_principaux, numero_chance_list[0]

def choisir_numeros_anti_popularite_lambda(config, df_historique=None, top_n_sums_to_avoid=5):
    print("\n[STRATEGIE] Génération d'une combinaison 'moins populaire'...")
    if df_historique is None or df_historique.empty:
        print("[ATTENTION] Pas de données historiques pour 'anti-popularité', génération aléatoire classique.")
        return generer_combinaison_aleatoire_lambda(config)

    num_cols = [f"N{i}" for i in range(1, config["numbers_to_choose"] + 1)]
    valid_num_cols = [col for col in num_cols if col in df_historique.columns]
    sommes_a_eviter = set()

    if valid_num_cols:
        try:
            df_historique_nums = df_historique[valid_num_cols].apply(pd.to_numeric, errors='coerce').dropna()
            if not df_historique_nums.empty:
                sommes_historiques = df_historique_nums.sum(axis=1)
                if not sommes_historiques.empty:
                    frequence_sommes = Counter(sommes_historiques)
                    sommes_a_eviter = {somme for somme, count in frequence_sommes.most_common(top_n_sums_to_avoid)}
                    print(f"  Sommes de numéros principaux à éviter: {sommes_a_eviter if sommes_a_eviter else 'Aucune'}")
                else: print("[INFO] Pas de sommes historiques à analyser.")
            else: print("[ATTENTION] Pas de données numériques valides pour les sommes historiques.")
        except Exception as e: print(f"[ERREUR] Calcul des sommes historiques échoué: {e}.")

    tentatives = 0
    while tentatives < 100:
        principaux, chance = generer_combinaison_aleatoire_lambda(config)
        somme_principaux = sum(principaux)
        nb_bas = sum(1 for n in principaux if n <= 31) # Numéros "dates"
        if somme_principaux not in sommes_a_eviter and nb_bas <= (config["numbers_to_choose"] // 2 + 1) : # Eviter trop de "dates"
            print(f"  Combinaison 'moins populaire' générée: {principaux} + {chance} (Somme: {somme_principaux}, N° bas: {nb_bas})")
            return principaux, chance
        tentatives += 1
    print("[ATTENTION] N'a pas pu générer une combinaison 'moins populaire' distincte, retour à l'aléatoire.")
    return generer_combinaison_aleatoire_lambda(config)

def choisir_numeros_froids_lambda(config, frequences_principales=None, frequences_chance=None, nombre_a_considerer=15):
    print(f"\n[STRATEGIE] Choix de numéros 'froids' (basée sur les moins fréquents)...")
    principaux_froids = []
    if frequences_principales:
        tous_les_numeros_p = list(range(config["min_val_number"], config["total_numbers"] + 1))
        numeros_tries_par_frequence_p = sorted(tous_les_numeros_p, key=lambda x: frequences_principales.get(x, 0))
        candidats_froids_p = numeros_tries_par_frequence_p[:nombre_a_considerer]
        if len(candidats_froids_p) >= config["numbers_to_choose"]:
            principaux_froids = sorted(random.sample(candidats_froids_p, config["numbers_to_choose"]))
        else:
            print(f"[ATTENTION] Pas assez de numéros principaux 'froids' distincts ({len(candidats_froids_p)} trouvés), complétion aléatoire.")
            principaux_froids = sorted(list(set(candidats_froids_p))) # Prendre ce qu'on a
            needed = config["numbers_to_choose"] - len(principaux_froids)
            if needed > 0:
                pool_restant = [n for n in tous_les_numeros_p if n not in principaux_froids]
                if len(pool_restant) >= needed:
                    principaux_froids.extend(random.sample(pool_restant, needed))
                else: # Si toujours pas assez, prendre tout ce qui reste
                    principaux_froids.extend(pool_restant)
                principaux_froids = sorted(principaux_froids)
                if len(principaux_froids) < config["numbers_to_choose"]: # Ultime vérification
                    print(f"[ATTENTION] Impossible de former une combinaison complète de numéros froids. Numéros actuels: {principaux_froids}")
    if not principaux_froids or len(principaux_froids) < config["numbers_to_choose"]:
        print("[ATTENTION] Impossible de déterminer les numéros principaux 'froids' ou pas assez, génération aléatoire.")
        principaux_froids, _ = generer_combinaison_aleatoire_lambda(config) # Retourne une combinaison complète

    chance_froide = None
    if frequences_chance:
        tous_les_numeros_c = list(range(config["min_val_lucky"], config["total_lucky_numbers"] + 1))
        numeros_tries_par_frequence_c = sorted(tous_les_numeros_c, key=lambda x: frequences_chance.get(x, 0))
        candidats_froids_c = numeros_tries_par_frequence_c[:max(1, config["lucky_numbers_to_choose"] + 2)] # Un peu plus de choix
        if candidats_froids_c:
            chance_froide = random.choice(candidats_froids_c)
    if chance_froide is None:
        print("[ATTENTION] Impossible de déterminer un N° Chance 'froid', génération aléatoire.")
        _, chance_froide = generer_combinaison_aleatoire_lambda(config)

    print(f"  Combinaison 'froide' suggérée: {principaux_froids} + {chance_froide}")
    return principaux_froids, chance_froide

def enregistrer_jeu_lambda(date_str, numeros_principaux, numero_chance, mise=2.50):
    print(f"\n[JEU ENREGISTRÉ] Date: {date_str}, Numéros: {numeros_principaux}, Chance: {numero_chance}, Mise: {mise}€")

def strategie_loto_lambda_main():
    print("Bienvenue dans l'Assistant Loto pour Personne Lambda !")
    calculer_probabilites_lambda(CONFIG_LOTO_LAMBDA)

    df_historique_lambda = fetch_lotto_data()
    frequences_principales_lambda, frequences_chance_lambda = None, None
    if df_historique_lambda is not None and not df_historique_lambda.empty:
        frequences_principales_lambda, frequences_chance_lambda = analyser_donnees_historiques_simplifie_lambda(df_historique_lambda, CONFIG_LOTO_LAMBDA)
    else:
        print("[INFO] Aucune donnée historique chargée pour les analyses détaillées du mode lambda.")

    print("\n" + "-" * 40)
    print("--- Choix de la Stratégie de Sélection des Numéros ---")
    print("1: Combinaison complètement aléatoire")
    print("2: Combinaison 'moins populaire' (basée sur sommes et peu de numéros 'date')")
    print("3: Combinaison de numéros 'froids' (basée sur les moins fréquents)")
    print("-" * 40)
    choix_strategie = input("Entrez votre choix de stratégie (1, 2, ou 3): ")

    numeros_suggeres_p, numero_suggere_c = [], None

    if choix_strategie == '1':
        numeros_suggeres_p, numero_suggere_c = generer_combinaison_aleatoire_lambda(CONFIG_LOTO_LAMBDA)
        print(f"\n  Combinaison aléatoire suggérée : {numeros_suggeres_p} + {numero_suggere_c}")
    elif choix_strategie == '2':
        numeros_suggeres_p, numero_suggere_c = choisir_numeros_anti_popularite_lambda(CONFIG_LOTO_LAMBDA, df_historique_lambda)
    elif choix_strategie == '3':
        numeros_suggeres_p, numero_suggere_c = choisir_numeros_froids_lambda(CONFIG_LOTO_LAMBDA, frequences_principales_lambda, frequences_chance_lambda)
    else:
        print("\nChoix invalide. Génération d'une combinaison aléatoire par défaut.")
        numeros_suggeres_p, numero_suggere_c = generer_combinaison_aleatoire_lambda(CONFIG_LOTO_LAMBDA)
        print(f"  Combinaison aléatoire suggérée : {numeros_suggeres_p} + {numero_suggere_c}")

    if numeros_suggeres_p and numero_suggere_c is not None:
        date_du_jour_str = pd.Timestamp.now().strftime("%Y-%m-%d")
        enregistrer_jeu_lambda(date_du_jour_str, numeros_suggeres_p, numero_suggere_c)

    print("\n" + "="*40)
    print("N'oubliez pas de jouer de manière responsable et de respecter votre budget !")
    print("Cet algorithme est un outil d'exploration et de divertissement, il ne garantit aucun gain.")
    print("="*40)


# --- NOUVELLE Fonction : Modélisation des métriques de séries temporelles ---
def analyze_timeseries_metric_models(df_results_time):
    print(f"\n{'='*40}")
    print(f" MODÉLISATION DES MÉTRIQUES DE FENÊTRES GLISSANTES (ARIMA)")
    print(f" Hypothèse: les structures ACF/PACF ne sont pas (uniquement) des artefacts du fenêtrage.")
    print(f"{'='*40}")

    metrics_to_model = {
        'compagnons_specifiques_count': (1, 0, 0),
        'ecart_moyen_global_fenetre': (0, 0, 1),
        'somme_moyenne_fenetre': (1, 0, 0),
        'variance_numeros_fenetre': (1, 0, 0)
    }

    for metric_col, order in metrics_to_model.items():
        if metric_col in df_results_time.columns and df_results_time[metric_col].notna().sum() > 20:
            print(f"\n--- Modélisation ARIMA pour '{metric_col}' ---")
            series = df_results_time[metric_col].dropna()

            if len(series) <= 20:
                print(f"Pas assez de points de données ({len(series)}) pour modéliser '{metric_col}' après suppression des NaN.")
                continue

            model_fit_final = None
            final_order_for_plot = order

            if metric_col == 'compagnons_specifiques_count':
                print(f"--- Recherche auto ARIMA pour '{metric_col}' ---")
                try:
                    auto_model = pm.auto_arima(
                        series,
                        start_p=0, start_q=0,
                        max_p=3, max_q=3,
                        d=None,
                        seasonal=False,
                        stepwise=True,
                        suppress_warnings=True,
                        error_action='ignore',
                        trace=False
                    )
                    print(f"Meilleur ordre ARIMA trouvé par auto_arima pour '{metric_col}': {auto_model.order}")
                    model_fit_final = auto_model
                    final_order_for_plot = auto_model.order
                except Exception as e_auto_arima:
                    print(f"  Erreur lors de auto_arima pour '{metric_col}': {e_auto_arima}. Utilisation de l'ordre manuel {order}.")

            if model_fit_final is None:
                print(f"--- Utilisation de l'ordre manuel ARIMA {order} pour '{metric_col}' ---")
                try:
                    model = ARIMA(series, order=order)
                    model_fit_final = model.fit()
                except Exception as e_manual_arima:
                    print(f"  Erreur lors de l'ajustement du modèle ARIMA manuel {order} pour '{metric_col}': {e_manual_arima}")
                    continue

            # Extraction des résidus et vérification
            residuals = getattr(model_fit_final, 'resid', None)
            if residuals is None:
                print(f"  Impossible d'obtenir les résidus pour '{metric_col}'. Modèle non compatible ?")
                continue

            if callable(residuals):
                try:
                    residuals = residuals()
                except Exception as e:
                    print(f"  Les résidus pour '{metric_col}' sont une méthode non appelable ou échouent à l'appel : {e}")
                    continue

            if not isinstance(residuals, (np.ndarray, pd.Series, list)):
                print(f"  Les résidus pour '{metric_col}' ne sont pas un tableau de nombres : {type(residuals)}")
                continue
            if len(residuals) == 0:
                print(f"  Pas de résidus à tracer pour {metric_col}")
                continue

            os.makedirs("plots", exist_ok=True)
            fig_res, axes_res = plt.subplots(3, 1, figsize=(12, 10))
            axes_res[0].plot(residuals)
            axes_res[0].set_title(f'Résidus du modèle ARIMA{final_order_for_plot} pour {metric_col}')
            axes_res[0].grid(True)

            lags_to_plot_acf_pacf = 0
            if len(residuals) > 3:
                lags_to_plot_acf_pacf = min(20, len(residuals) // 2 - 1)
            if lags_to_plot_acf_pacf > 0:
                sm.graphics.tsa.plot_acf(residuals, lags=lags_to_plot_acf_pacf, ax=axes_res[1])
                axes_res[1].set_title(f'ACF des Résidus pour {metric_col}')
                axes_res[1].grid(True)
                sm.graphics.tsa.plot_pacf(residuals, lags=lags_to_plot_acf_pacf, ax=axes_res[2], method='ywm')
                axes_res[2].set_title(f'PACF des Résidus pour {metric_col}')
                axes_res[2].grid(True)
            else:
                axes_res[1].set_title(f'ACF des Résidus pour {metric_col} (non tracé - pas assez de données)')
                axes_res[2].set_title(f'PACF des Résidus pour {metric_col} (non tracé - pas assez de données)')

            plt.tight_layout()
            order_str_filename = '_'.join(map(str, final_order_for_plot))
            plt.savefig(f"plots/arima_residuals_{metric_col}_order_{order_str_filename}.png")
            plt.close()
            print(f"  Graphique des résidus et ACF/PACF des résidus pour '{metric_col}' (ordre {final_order_for_plot}) sauvegardé.")

            try:
                from statsmodels.stats.diagnostic import acorr_ljungbox
                lags_ljungbox = None
                if len(residuals) > 1:
                    lags_ljungbox_val = min(10, len(residuals) - 1)
                    if lags_ljungbox_val > 0:
                        lags_ljungbox = [lags_ljungbox_val]
                if lags_ljungbox:
                    ljung_box_result = acorr_ljungbox(residuals, lags=lags_ljungbox, return_df=True)
                    print(f"\n  Test de Ljung-Box sur les résidus pour '{metric_col}':")
                    print(ljung_box_result)
                    if not ljung_box_result.empty and ljung_box_result['lb_pvalue'].iloc[0] > 0.05:
                        print(f"  Conclusion Ljung-Box: Les résidus pour '{metric_col}' semblent être indépendants (p-valeur > 0.05). Le modèle capture bien la structure.")
                    elif not ljung_box_result.empty:
                        print(f"  Conclusion Ljung-Box: Il reste de l'autocorrélation dans les résidus pour '{metric_col}' (p-valeur <= 0.05). Le modèle pourrait être amélioré.")
                else:
                    print(f"  Pas assez de résidus ({len(residuals)}) pour le test de Ljung-Box pour {metric_col}.")
            except Exception as e_lb:
                print(f"  Erreur lors du test de Ljung-Box pour '{metric_col}': {e_lb}")
        else:
            print(f"\nPas assez de données ou métrique '{metric_col}' non trouvée pour la modélisation.")


# --- NOUVELLE Fonction : Analyse sur Fenêtres Glissantes ---
def analyze_sliding_windows(lotto_data_asc, window_size=100, step_size=20):
    if not RUN_SLIDING_WINDOW_ENHANCED_METRICS:
        print("Analyse sur fenêtres glissantes (métriques étendues) désactivée.")
        return

    print(f"\n" + "="*40)
    print(f" ANALYSE SUR FENÊTRES GLISSANTES (taille={window_size}, pas={step_size})")
    print(f"="*40)
    if lotto_data_asc.empty or len(lotto_data_asc) < window_size:
        print("Pas assez de données pour l'analyse sur fenêtres glissantes.")
        return

    num_cols = [f"N{i}" for i in range(1,6)]
    results_over_time = []

    vos_paires_compagnons = set()
    if len(MES_NUMEROS_PRINCIPAUX_JOUES) >= 2:
        for paire in combinations(sorted(list(MES_NUMEROS_PRINCIPAUX_JOUES)), 2):
            vos_paires_compagnons.add(paire)

    for i in range(0, len(lotto_data_asc) - window_size + 1, step_size):
        window_df = lotto_data_asc.iloc[i : i + window_size]
        window_end_date = window_df['Date'].iloc[-1]

        window_numbers_principaux_list = []
        for _, row in window_df[num_cols].iterrows():
            try:
                window_numbers_principaux_list.append([int(n) for n in row.dropna() if pd.notna(n) and str(n).isdigit()])
            except ValueError: continue

        if not window_numbers_principaux_list: continue
        all_numbers_flat_window = [num for sublist in window_numbers_principaux_list for num in sublist]
        if not all_numbers_flat_window: continue
        freq_window = Counter(all_numbers_flat_window)

        freq_6_w = freq_window.get(6, 0)
        freq_7_w = freq_window.get(7, 0) # Ajout pour le numéro 7 si vous le suivez
        freq_9_w = freq_window.get(9, 0)
        freq_30_w = freq_window.get(30, 0)
        var_nums_w = np.var(all_numbers_flat_window) if all_numbers_flat_window else np.nan
        std_nums_w = np.std(all_numbers_flat_window) if all_numbers_flat_window else np.nan

        compagnons_specifiques_count_w = 0
        for draw_nums in window_numbers_principaux_list:
            for paire_specifique in vos_paires_compagnons:
                if set(paire_specifique).issubset(set(draw_nums)):
                    compagnons_specifiques_count_w += 1

        gaps_data_w = {num: [] for num in range(1, 50)}
        last_seen_index_w = {}
        for current_idx_w, draw_nums_w in enumerate(window_numbers_principaux_list):
            for num_val_w in range(1, 50):
                if num_val_w in draw_nums_w:
                    if num_val_w in last_seen_index_w:
                        gaps_data_w[num_val_w].append(current_idx_w - last_seen_index_w[num_val_w])
                    last_seen_index_w[num_val_w] = current_idx_w
        all_gaps_flat_w = [g for gap_list in gaps_data_w.values() for g in gap_list if gap_list]
        overall_avg_gap_w = np.mean(all_gaps_flat_w) if all_gaps_flat_w else np.nan
        avg_sum_w = np.mean([sum(d) for d in window_numbers_principaux_list]) if window_numbers_principaux_list else np.nan

        results_over_time.append({
            'date_fin_fenetre': window_end_date,
            'freq_6': freq_6_w, 'freq_7': freq_7_w, 'freq_9': freq_9_w, 'freq_30': freq_30_w, # freq_7 ajoutée
            'ecart_moyen_global_fenetre': overall_avg_gap_w,
            'somme_moyenne_fenetre': avg_sum_w,
            'variance_numeros_fenetre': var_nums_w,
            'std_numeros_fenetre': std_nums_w,
            'compagnons_specifiques_count': compagnons_specifiques_count_w
        })

    if not results_over_time:
        print("Aucun résultat généré par l'analyse sur fenêtres glissantes.")
        return

    df_results_time = pd.DataFrame(results_over_time)
    df_results_time.set_index('date_fin_fenetre', inplace=True)

    print("\nQuelques résultats de l'analyse sur fenêtres glissantes (début et fin) :")
    print(df_results_time.head(3).to_string())
    print("...")
    print(df_results_time.tail(3).to_string())

    if RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS:
        num_metrics_to_plot = 5
        fig, axes = plt.subplots(num_metrics_to_plot, 1, figsize=(15, num_metrics_to_plot * 4), sharex=True)
        # Ajustement pour inclure freq_7 si elle est pertinente
        freq_cols_to_plot = ['freq_6', 'freq_9', 'freq_30']
        if 'freq_7' in df_results_time.columns: # Vérifier si la colonne existe
            freq_cols_to_plot.append('freq_7')
        df_results_time[freq_cols_to_plot].plot(ax=axes[0], marker='o', linestyle='-')

        axes[0].set_title('Fréquence de vos numéros sur fenêtres glissantes')
        axes[0].set_ylabel('Fréquence'); axes[0].grid(True); axes[0].legend()

        df_results_time['ecart_moyen_global_fenetre'].plot(ax=axes[1], marker='o', linestyle='-', color='green')
        axes[1].axhline(y=8.8, color='r', linestyle='--', label='Ecart théorique (8.8)')
        axes[1].set_title('Écart moyen global sur fenêtres glissantes')
        axes[1].set_ylabel('Écart Moyen'); axes[1].legend(); axes[1].grid(True)

        df_results_time['somme_moyenne_fenetre'].plot(ax=axes[2], marker='o', linestyle='-', color='purple')
        axes[2].axhline(y=125, color='r', linestyle='--', label='Somme théorique moyenne (approx 125)')
        axes[2].set_title('Somme moyenne des numéros sur fenêtres glissantes')
        axes[2].set_ylabel('Somme Moyenne'); axes[2].legend(); axes[2].grid(True)

        df_results_time['variance_numeros_fenetre'].plot(ax=axes[3], marker='o', linestyle='-', color='orange')
        axes[3].set_title('Variance des numéros sur fenêtres glissantes')
        axes[3].set_ylabel('Variance'); axes[3].grid(True); axes[3].legend()

        df_results_time['compagnons_specifiques_count'].plot(ax=axes[4], marker='o', linestyle='-', color='brown')
        axes[4].set_title('Nombre de vos paires de compagnons par fenêtre')
        axes[4].set_ylabel('Compte'); axes[4].grid(True); axes[4].legend()

        plt.xlabel('Date de Fin de Fenêtre'); plt.tight_layout()
        os.makedirs("plots", exist_ok=True); plt.savefig("plots/analyse_fenetres_glissantes_etendue.png"); plt.close()
        print("Graphique de l'analyse étendue sur fenêtres glissantes sauvegardé.")

        if RUN_SLIDING_WINDOW_TIMESERIES_ANALYSIS:
            print("\n--- Analyse ACF/PACF sur métriques de fenêtres glissantes ---")
            metrics_to_analyze_ts = ['ecart_moyen_global_fenetre', 'somme_moyenne_fenetre', 'variance_numeros_fenetre', 'compagnons_specifiques_count']
            for metric_col in metrics_to_analyze_ts:
                if metric_col in df_results_time.columns and df_results_time[metric_col].notna().sum() > 21: # Assez de points pour ACF/PACF
                    series_for_acf = df_results_time[metric_col].dropna()
                    if len(series_for_acf) > 3: # Minimum pour tracer
                        try:
                            fig_ts, axes_ts = plt.subplots(1,2,figsize=(12,4))
                            lags_acf_pacf = min(21, len(series_for_acf)//2 -1) if len(series_for_acf)//2 -1 > 0 else 1

                            sm.graphics.tsa.plot_acf(series_for_acf, lags=lags_acf_pacf, ax=axes_ts[0])
                            sm.graphics.tsa.plot_pacf(series_for_acf, lags=lags_acf_pacf, ax=axes_ts[1], method='ywm')
                            fig_ts.suptitle(f'ACF et PACF pour {metric_col}')
                            plt.tight_layout(rect=[0, 0.03, 1, 0.95])
                            plt.savefig(f"plots/acf_pacf_{metric_col}.png"); plt.close()
                            print(f"  Graphique ACF/PACF pour '{metric_col}' sauvegardé.")
                        except Exception as e_ts:
                            print(f"  Erreur lors de l'analyse ACF/PACF pour '{metric_col}': {e_ts}")
                    else:
                        print(f"  Pas assez de données pour ACF/PACF de '{metric_col}' après dropna ({len(series_for_acf)} points).")


            # Appel à la nouvelle fonction de modélisation
            analyze_timeseries_metric_models(df_results_time.copy()) # Utiliser une copie pour éviter modifications


# --- Le reste de votre script continue ici...
# --- NOUVELLE Fonction : Analyse des Transitions ---
def analyze_transitions(lotto_data_asc, lag=1, top_n=10):
    if not RUN_TRANSITION_PROBABILITIES:
        print("Analyse des transitions désactivée.")
        return None

    print(f"\n" + "="*40); print(f" ANALYSE DES TRANSITIONS DE NUMÉROS (Lag={lag}, Top {top_n})"); print(f"="*40)
    if lotto_data_asc.empty or len(lotto_data_asc) <= lag: print("Pas assez de données pour l'analyse des transitions."); return None

    num_cols = [f"N{i}" for i in range(1, 6)]; transitions = Counter()
    from_counts = Counter()

    for i in range(len(lotto_data_asc) - lag):
        current_draw_nums_s = lotto_data_asc.iloc[i][num_cols].dropna()
        next_draw_nums_s = lotto_data_asc.iloc[i + lag][num_cols].dropna()
        try:
            current_draw_nums = {int(n) for n in current_draw_nums_s}
            next_draw_nums = {int(n) for n in next_draw_nums_s}
        except ValueError: continue

        for num1 in current_draw_nums:
            from_counts[num1] += 1
            for num2 in next_draw_nums:
                transitions[(num1, num2)] += 1

    if not transitions: print("Aucune transition trouvée."); return None

    transition_data = []
    for (num_from, num_to), count in transitions.items():
        prob_transition = count / from_counts[num_from] if from_counts[num_from] > 0 else 0
        transition_data.append({'From': num_from, 'To': num_to, 'Count': count, 'Prob_To_Given_From': prob_transition})

    df_transitions = pd.DataFrame(transition_data)
    df_transitions.sort_values('Count', ascending=False, inplace=True)

    print(f"Top {top_n} transitions les plus fréquentes (Numéro -> Numéro après {lag} tirage(s)):")
    df_display = df_transitions.head(top_n).copy()
    df_display['P(To|From)'] = df_display['Prob_To_Given_From'].map('{:.4f}'.format)
    df_display['P(To)_baseline'] = (5/49)
    df_display['P(To)_baseline'] = df_display['P(To)_baseline'].map('{:.4f}'.format)
    print(df_display[['From', 'To', 'Count', 'P(To|From)', 'P(To)_baseline']].to_string(index=False))


    print(f"\nTransitions les plus fréquentes depuis VOS NUMÉROS ({MES_NUMEROS_PRINCIPAUX_JOUES}):")
    for num_perso in MES_NUMEROS_PRINCIPAUX_JOUES:
        top_transitions_from_perso = df_transitions[df_transitions['From'] == num_perso].sort_values('Count', ascending=False).head(min(5, top_n))
        if not top_transitions_from_perso.empty:
            print(f"  Depuis {num_perso}:")
            for _, row_trans in top_transitions_from_perso.iterrows():
                print(f"     -> {row_trans['To']}: {row_trans['Count']} fois (P={row_trans['Prob_To_Given_From']:.4f}, Baseline P(To)={(5/49):.4f})")
        else:
            print(f"  Aucune transition observée depuis {num_perso} avec les filtres actuels.")

    return df_transitions


# --- NOUVELLE Fonction : Clustering des Tirages ---
def get_draw_features(lotto_data, config):
    num_cols = [f"N{i}" for i in range(1, config["numbers_to_choose"] + 1)]
    draw_features = []
    original_indices = []

    for index, row in lotto_data.iterrows():
        try:
            numbers = sorted([int(row[n]) for n in num_cols if pd.notna(row[n])])
            if len(numbers) != config["numbers_to_choose"]: continue

            features = [
                sum(numbers), sum(1 for n in numbers if n % 2 == 0), numbers[0], numbers[-1], numbers[-1] - numbers[0],
                sum(1 for n in numbers if n <= 10), sum(1 for n in numbers if 11 <= n <= 20),
                sum(1 for n in numbers if 21 <= n <= 30), sum(1 for n in numbers if 31 <= n <= 40),
                sum(1 for n in numbers if 41 <= n <= 49),
            ]
            has_seq2, has_seq3 = 0, 0
            for i_seq in range(len(numbers) - 1):
                if numbers[i_seq+1] - numbers[i_seq] == 1:
                    has_seq2 = 1
                    if i_seq + 2 < len(numbers) and numbers[i_seq+2] - numbers[i_seq+1] == 1:
                        has_seq3 = 1; break
            features.extend([has_seq2, has_seq3])
            draw_features.append(features)
            original_indices.append(index)
        except (ValueError, TypeError): continue

    df_out = pd.DataFrame(draw_features, columns=[
        'somme', 'nb_pairs', 'min_num', 'max_num', 'etendue',
        'diz_1_10', 'diz_11_20', 'diz_21_30', 'diz_31_40', 'diz_41_49',
        'has_seq2', 'has_seq3'
    ])
    df_out.index = original_indices
    return df_out


def cluster_draws(lotto_data_asc, config, n_clusters_kmeans=5):
    if not RUN_DRAW_CLUSTERING: print("Clustering de tirages désactivé."); return None
    print(f"\n" + "="*40); print(f" CLUSTERING DES TIRAGES (K-Means avec {n_clusters_kmeans} clusters)"); print(f"="*40)

    if lotto_data_asc.empty or len(lotto_data_asc) < n_clusters_kmeans * 5:
        print("Pas assez de données pour le clustering."); return None

    df_features = get_draw_features(lotto_data_asc, config)
    if df_features.empty: print("Aucune feature extraite pour le clustering."); return None

    df_features.dropna(inplace=True)
    if df_features.empty: print("Aucune feature valide après nettoyage pour le clustering."); return None

    scaler = StandardScaler(); scaled_features = scaler.fit_transform(df_features)
    lotto_data_asc_clustered = lotto_data_asc.loc[df_features.index].copy()

    try:
        kmeans = KMeans(n_clusters=n_clusters_kmeans, random_state=42, n_init='auto')
        lotto_data_asc_clustered['cluster_kmeans'] = kmeans.fit_predict(scaled_features)

        print(f"\nRésultats K-Means:\n{lotto_data_asc_clustered['cluster_kmeans'].value_counts().sort_index()}")
        print("\nMoyenne des features par cluster K-Means:")

        df_features_for_groupby = df_features.copy()
        df_features_for_groupby['cluster_kmeans'] = lotto_data_asc_clustered['cluster_kmeans'].values
        feature_cols_for_mean = [col for col in df_features.columns if col != 'cluster_kmeans']
        print(df_features_for_groupby.groupby('cluster_kmeans')[feature_cols_for_mean].mean().round(2).to_string())

        if RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS and len(scaled_features) > 2:
            from sklearn.decomposition import PCA
            pca = PCA(n_components=2, random_state=42)
            reduced_features = pca.fit_transform(scaled_features)
            plt.figure(figsize=(10, 7))
            sns.scatterplot(x=reduced_features[:,0], y=reduced_features[:,1],
                            hue=lotto_data_asc_clustered['cluster_kmeans'].values,
                            palette='viridis', s=50, alpha=0.7)
            plt.title(f'Clusters de Tirages (K-Means via PCA)'); plt.xlabel('Composante Principale 1'); plt.ylabel('Composante Principale 2')
            plt.legend(title='Cluster K-Means'); plt.grid(True); plt.tight_layout()
            plt.savefig("plots/draw_clusters_kmeans_pca.png"); plt.close()
            print("Graphique clustering K-Means (PCA) sauvegardé.")
    except Exception as e_kmeans:
        print(f"Erreur K-Means: {e_kmeans}")
        if 'cluster_kmeans' in lotto_data_asc_clustered.columns: lotto_data_asc_clustered.drop('cluster_kmeans', axis=1, inplace=True)

    return lotto_data_asc_clustered

# --- NOUVELLE FONCTION : Analyse des séquences fréquentes avec PrefixSpan ---
def analyze_frequent_sequences_prefixspan(lotto_data, min_support_ratio=0.005, max_pattern_length=5):
    """
    Analyse les séquences de numéros fréquents dans les tirages du loto en utilisant PrefixSpan.

    Args:
        lotto_data (pd.DataFrame): DataFrame contenant les tirages du loto (colonnes N1 à N5).
        min_support_ratio (float): Ratio de support minimum (ex: 0.01 pour 1% des tirages).
        max_pattern_length (int): Longueur maximale des séquences à afficher.
    """
    if not isinstance(lotto_data, pd.DataFrame) or lotto_data.empty:
        print("Aucune donnée valide pour l'analyse PrefixSpan.")
        return

    print(f"\n" + "="*40)
    print(f" ANALYSE DES SÉQUENCES FRÉQUENTES AVEC PREFIXSPAN")
    print(f" Support Minimum: {min_support_ratio*100:.2f}% des tirages")
    print(f" Longueur Maximale de Séquence: {max_pattern_length}")
    print(f"="*40)

    num_cols = [f"N{i}" for i in range(1, 6)]
    if not all(c in lotto_data.columns for c in num_cols):
        print("Colonnes N1-N5 manquantes pour l'analyse PrefixSpan.")
        return

    sequences = []
    for _, row in lotto_data.iterrows():
        try:
            # Assurez-vous que les numéros sont des entiers et triés pour des séquences cohérentes
            current_draw_nums = sorted([int(n) for n in row[num_cols].dropna() if pd.notna(n) and str(n).isdigit()])
            if len(current_draw_nums) == 5: # Ne considérer que les tirages complets de 5 numéros
                sequences.append(current_draw_nums)
        except ValueError:
            logging.warning(f"Ligne avec numéros non convertibles (PrefixSpan): {row.tolist()}. Ignorée.")
            continue

    if not sequences:
        print("Aucune séquence valide extraite pour l'analyse PrefixSpan.")
        return

    # Calculer min_support en nombre absolu de séquences
    num_sequences = len(sequences)
    min_support_count = int(num_sequences * min_support_ratio)
    if min_support_count == 0 and num_sequences > 0: # S'assurer d'au moins 1 si le ratio est très petit
        min_support_count = 1

    print(f"Nombre total de tirages (séquences) pour PrefixSpan: {num_sequences}")
    print(f"Nombre minimum de support requis: {min_support_count} tirages")

    try:
        ps = PrefixSpan(sequences)
        # La méthode 'frequent' prend le support minimum directement comme premier argument.
        all_frequent_patterns_with_support = ps.frequent(min_support_count)

        # Trier les motifs par support (fréquence) dans l'ordre décroissant
        all_frequent_patterns_with_support.sort(key=lambda x: x[0], reverse=True)

        print("\nSéquences fréquentes (Support, Séquence):")
        found_patterns = False
        for support, pattern in all_frequent_patterns_with_support:
            # Filtrer par la longueur maximale souhaitée
            if len(pattern) <= max_pattern_length:
                print(f"  Support: {support} ({support/num_sequences*100:.2f}%), Séquence: {pattern}")
                found_patterns = True
        if not found_patterns:
            print("Aucune séquence fréquente trouvée avec les critères donnés.")

    except Exception as e:
        logging.error(f"Erreur lors de l'exécution de PrefixSpan: {e}", exc_info=True)
        print(f"Une erreur est survenue lors de l'analyse PrefixSpan: {e}")

    print(f"="*40)

# --- Fonctions d'Analyse Statistique (existantes) ---
def analyze_frequencies(lotto_data, title="Générale", save_prefix="general"):
    if lotto_data.empty:
        print(f"Aucune donnée pour analyse de fréquence '{title}'.")
        return None
    num_cols = [f"N{i}" for i in range(1,6)]
    if not all(c in lotto_data.columns for c in num_cols):
        print(f"Colonnes N1-N5 manquantes pour l'analyse de fréquence '{title}'.")
        return None

    valid_data = lotto_data[num_cols].dropna().copy()
    for col in num_cols: valid_data[col] = pd.to_numeric(valid_data[col], errors='coerce')
    all_numbers = valid_data.stack().dropna().astype(int).values

    if not all_numbers.size:
        print(f"Aucun numéro trouvé pour l'analyse de fréquence '{title}'.")
        return None

    max_num = 49
    freq = pd.Series(all_numbers).value_counts().reindex(range(1, max_num + 1), fill_value=0)

    if RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS:
        print(f"\n--- Fréquence des numéros ({title}) ---\nTop 10:\n{freq.sort_values(ascending=False).head(10)}")
        print("\nFréquences de VOS numéros joués (principaux) :")
        for num_perso in sorted(list(MES_NUMEROS_PRINCIPAUX_JOUES)):
            print(f"  Numéro {num_perso}: {freq.get(num_perso, 0)} apparitions")

    if (total_occ := len(all_numbers)) > 0 and (num_outcomes := max_num) > 0:
        exp_freq_val = total_occ / num_outcomes
        exp_freqs = np.full(num_outcomes, exp_freq_val)
        obs_freqs = freq.values
        if len(obs_freqs) == len(exp_freqs) > 1:
            chi2, p_val = chisquare(f_obs=obs_freqs, f_exp=exp_freqs)
            if RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS:
                print(f"Test chi² d'uniformité ({title}): stat={chi2:.2f}, p-val={p_val:.5f}. {'Distribution semble uniforme' if p_val >= 0.05 else 'Distribution NON uniforme (biais possible)'}")
        elif RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS:
            print(f"Impossible d'effectuer le test chi² ({title}): longueurs incompatibles ou < 2.")

    if RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS:
        plt.figure(figsize=(12,5))
        sns.barplot(x=freq.index, y=freq.values)
        plt.title(f"Fréquence des Numéros ({title})")
        plt.xlabel("Numéro"); plt.ylabel("Nombre d'apparitions")
        plt.xticks(np.arange(0, max_num + 1, 2))
        plt.grid(axis='y',linestyle='--'); plt.tight_layout();
        os.makedirs("plots", exist_ok=True); plt.savefig(f"plots/freq_histogram_{save_prefix}.png"); plt.close();
        print(f"Histogramme des fréquences sauvegardé: plots/freq_histogram_{save_prefix}.png")
    return freq

def analyze_frequencies_by_day(lotto_data):
    if not RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS: return
    print("\n" + "="*40 + "\n ANALYSE DES FRÉQUENCES PAR JOUR DE TIRAGE \n" + "="*40)
    if 'Jour' not in lotto_data.columns: print("La colonne 'Jour' est manquante."); return

    jours_disponibles = lotto_data["Jour"].astype(str).str.capitalize().unique()
    jours_a_analyser = [j for j in ['Lundi', 'Mercredi', 'Samedi'] if j in jours_disponibles]
    if not jours_a_analyser: print("Aucun des jours standards trouvé."); return

    freq_by_day, nums_by_day = {}, {}
    num_cols = [f"N{i}" for i in range(1,6)]
    if not all(c in lotto_data.columns for c in num_cols): print("Colonnes N1-N5 manquantes."); return

    for jour in jours_a_analyser:
        data_j = lotto_data[lotto_data["Jour"].astype(str).str.contains(jour, case=False, na=False)]
        if data_j.empty: print(f"Aucun tirage pour {jour}."); continue
        if RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS: print(f"\nAnalyse pour le jour : {jour}")
        freq_by_day[jour] = analyze_frequencies(data_j, title=f"Fréquences {jour} (données filtrées)", save_prefix=f"freq_{jour.lower()}_filtered")
        if freq_by_day[jour] is not None:
            valid_data_j = data_j[num_cols].dropna().copy()
            for col in num_cols: valid_data_j[col] = pd.to_numeric(valid_data_j[col], errors='coerce')
            nums_by_day[jour] = valid_data_j.stack().dropna().astype(int).values

    if len(nums_by_day) < 2: print("Pas assez de jours pour test d'indépendance."); return

    print("\n--- Test Chi² d'Indépendance des Distributions de Fréquences par Jour ---")
    all_possible_nums = range(1, 50)
    jours_data_valides = [j for j in jours_a_analyser if j in nums_by_day and nums_by_day[j].size > 0]
    if len(jours_data_valides) < 2: print("Moins de deux jours avec données suffisantes pour test."); return

    contingency_table = pd.DataFrame(index=all_possible_nums, columns=jours_data_valides, dtype=float)
    for num in all_possible_nums:
        for jour in jours_data_valides:
            contingency_table.loc[num, jour] = np.sum(nums_by_day[jour] == num)
    contingency_table = contingency_table.fillna(0).astype(float)

    observed_df_filtered = contingency_table.loc[(contingency_table.sum(axis=1) > 0)]
    if observed_df_filtered.shape[0] < 2 or observed_df_filtered.shape[1] < 2: print("Table de contingence invalide après filtrage."); return

    try:
        chi2, p_val, dof, expected = chi2_contingency(observed_df_filtered)
        print(f"Résultat Test Chi² d'indépendance: Chi²={chi2:.2f}, p-valeur={p_val:.5f}, ddl={dof}.")
        print(f"Conclusion: {'Distributions indépendantes du jour.' if p_val >=0.05 else 'Possible dépendance au jour.'}")
    except ValueError as e: print(f"Erreur calcul Chi² d'indépendance: {e}")

    if len(freq_by_day) > 1 and RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS:
        plt.figure(figsize=(14,7)); plotted_something = False
        for jour, freq_data in freq_by_day.items():
            if freq_data is not None and not freq_data.empty: sns.lineplot(x=freq_data.index, y=freq_data.values, label=jour); plotted_something=True
        if plotted_something:
            plt.title("Comparaison Fréquences par Jour"); plt.xlabel("Numéro"); plt.ylabel("Apparitions")
            plt.xticks(np.arange(1,50,2)); plt.legend(); plt.grid(axis='y',linestyle='--'); plt.tight_layout();
            plt.savefig("plots/freq_comparison_by_day_filtered.png"); plt.close();
            print("Graphique comparaison fréquences par jour sauvegardé.")

def analyze_combinations(lotto_data, n_numbers=2, max_num=49, top_n=21, save_prefix="pairs"):
    if not RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS and n_numbers > 2 : return None
    if lotto_data.empty: print(f"Aucune donnée pour combinaisons de {n_numbers}."); return None

    num_cols = [f"N{i}" for i in range(1,6)]
    if not all(c in lotto_data.columns for c in num_cols) or len(num_cols) < n_numbers: print(f"Colonnes N1-N5 manquantes/insuffisantes."); return None

    draws = []
    for _, row in lotto_data[num_cols].iterrows():
        try:
            valid_numbers_in_row = [int(n) for n in row.dropna() if pd.notna(n) and str(n).isdigit()]
            if len(valid_numbers_in_row) >= n_numbers: draws.append(valid_numbers_in_row)
        except ValueError: logging.warning(f"Ligne avec numéros non convertibles (combinations): {row.tolist()}"); continue

    if not draws: print(f"Aucun tirage valide pour combinaisons de {n_numbers}."); return None

    combos = [tuple(sorted(c)) for draw in draws for c in combinations(draw, n_numbers)]
    if not combos: print(f"Aucune combinaison de {n_numbers} générée."); return None

    counts = Counter(combos)
    df_combinations = pd.DataFrame.from_dict(counts, orient='index', columns=['Frequency']).sort_values('Frequency', ascending=False)

    if RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS:
        print(f"\n--- Fréquence des Combinaisons de {n_numbers} numéros ---\nTop {top_n}:\n{df_combinations.head(top_n).to_string()}")
        total_possible_combinations = nCr(max_num, n_numbers)
        print(f"Combinaisons possibles de {n_numbers} (sur {max_num}): {total_possible_combinations}")
        print(f"Combinaisons de {n_numbers} observées: {len(combos)}")
        if total_possible_combinations > 0: print(f"Fréquence moyenne attendue: {len(combos)/total_possible_combinations:.2f}")

        if not df_combinations.empty:
            plt.figure(figsize=(max(10, int(top_n*0.4)),6))
            top_plot_df = df_combinations.head(top_n)
            top_plot_df.index = top_plot_df.index.map(lambda x: ', '.join(map(str,x)) if isinstance(x, tuple) else x)
            sns.barplot(data=top_plot_df, x=top_plot_df.index, y='Frequency', hue=top_plot_df.index, palette='viridis', legend=False, dodge=False)
            plt.title(f"Top {top_n} Combinaisons de {n_numbers} Numéros"); plt.xlabel(f"Combinaison de {n_numbers}"); plt.ylabel("Fréquence")
            plt.xticks(rotation=45, ha='right'); plt.tight_layout();
            os.makedirs("plots", exist_ok=True); plt.savefig(f"plots/top_{top_n}_{save_prefix}_combinations.png"); plt.close();
            print(f"Graphique top {top_n} {save_prefix} sauvegardé.")
    return df_combinations

def analyze_gaps(lotto_data_asc, max_num=49):
    if not RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS: return None
    print("\n" + "="*40 + "\n ANALYSE DES ÉCARTS (GAPS) ENTRE APPARITIONS \n" + "="*40)
    if lotto_data_asc.empty: print("Aucune donnée pour analyse des écarts."); return None

    num_cols = [f"N{i}" for i in range(1,6)]
    if not all(c in lotto_data_asc.columns for c in num_cols): print("Colonnes N1-N5 manquantes."); return None

    draws_list = []
    for _, row in lotto_data_asc[num_cols].iterrows():
        try: draws_list.append([int(n) for n in row.dropna() if pd.notna(n) and str(n).isdigit()])
        except ValueError: logging.warning(f"Ligne avec numéros non convertibles (gaps): {row.tolist()}"); continue

    if not draws_list: print("Aucun tirage valide pour analyse des écarts."); return None

    last_seen_index, gaps_data = {}, {num: [] for num in range(1, max_num + 1)}
    for current_index, draw_numbers in enumerate(draws_list):
        for num_val in range(1, max_num + 1):
            if num_val in draw_numbers:
                if num_val in last_seen_index: gaps_data[num_val].append(current_index - last_seen_index[num_val])
                last_seen_index[num_val] = current_index

    average_gaps = {num: np.mean(gap_list) if gap_list else np.nan for num, gap_list in gaps_data.items()}

    if RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS:
        print("Écart moyen entre les apparitions pour chaque numéro (1-49):")
        for num, avg_gap_val in sorted(average_gaps.items()): print(f"  Numéro {num:2d}: {avg_gap_val:.2f} tirages" if pd.notna(avg_gap_val) else f"  Numéro {num:2d}: N/A")

    all_gaps_flat_list = [g for gap_list in gaps_data.values() for g in gap_list if gap_list]
    overall_avg_gap = np.mean(all_gaps_flat_list) if all_gaps_flat_list else np.nan
    expected_gap = (max_num / 5.0) - 1.0

    if RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS:
        print(f"\nÉcart moyen global observé: {overall_avg_gap:.2f} tirages.")
        print(f"Écart moyen attendu (théorique): {expected_gap:.2f} tirages.")

    if RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS:
        plot_numbers = sorted([n for n, avg in average_gaps.items() if pd.notna(avg)])
        plot_values = [average_gaps[n] for n in plot_numbers]
        if plot_numbers:
            plt.figure(figsize=(12,5))
            sns.barplot(x=plot_numbers, y=plot_values, color='skyblue')
            if pd.notna(expected_gap): plt.axhline(expected_gap, color='red', linestyle='--', label=f'Écart Attendu ({expected_gap:.2f})')
            plt.title("Écart Moyen Entre Apparitions par Numéro"); plt.xlabel("Numéro"); plt.ylabel("Écart Moyen")
            tick_step = max(1, len(plot_numbers) // 21)
            plt.xticks(ticks=np.arange(0, len(plot_numbers), tick_step), labels=[plot_numbers[i] for i in np.arange(0, len(plot_numbers), tick_step).astype(int)] if plot_numbers else [])
            plt.legend(); plt.grid(axis='y',linestyle='--'); plt.tight_layout();
            os.makedirs("plots", exist_ok=True); plt.savefig(f"plots/average_gaps_by_number.png"); plt.close();
            print("Graphique écarts moyens par numéro sauvegardé.")

        example_numbers_for_gaps = [n for n in [1,6,7,9,13,14,19,21,25,30,31,35,37,42,43,49] if n in gaps_data and len(gaps_data[n]) > 5]
        if example_numbers_for_gaps:
            num_examples = len(example_numbers_for_gaps); ncols_plot = min(3, num_examples); nrows_plot = (num_examples + ncols_plot - 1) // ncols_plot
            if nrows_plot > 0:
                fig, axes_array = plt.subplots(nrows_plot, ncols_plot, figsize=(ncols_plot*4, nrows_plot*3.5), squeeze=False)
                axes_flat = axes_array.flatten()
                for i, num_example in enumerate(example_numbers_for_gaps):
                    if i < len(axes_flat) and gaps_data[num_example]:
                        sns.histplot(gaps_data[num_example], bins=max(1, (max(gaps_data[num_example]) if gaps_data[num_example] else 1)//2), kde=False, ax=axes_flat[i])
                        axes_flat[i].set_title(f"Distribution Écarts N°{num_example}"); axes_flat[i].set_xlabel("Écart"); axes_flat[i].set_ylabel("Fréquence"); axes_flat[i].grid(axis='y',linestyle='--')
                plt.tight_layout(); os.makedirs("plots", exist_ok=True); plt.savefig("plots/gap_distributions_examples.png"); plt.close();
                print("Graphique distributions d'écarts (exemples) sauvegardé.")
    return gaps_data

def analyze_temporal_correlation(lotto_data_asc, max_lag=10, max_num=49):
    if not RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS: return None
    print("\n" + "="*40 + "\n ANALYSE DE L'AUTOCORRÉLATION TEMPORELLE \n" + "="*40)
    if lotto_data_asc.empty: print("Aucune donnée pour corrélation temporelle."); return None

    num_cols = [f"N{i}" for i in range(1,6)];
    if not all(c in lotto_data_asc.columns for c in num_cols): print("Colonnes N1-N5 manquantes."); return None

    draws_list_int = []
    for _, row in lotto_data_asc[num_cols].iterrows():
        try: draws_list_int.append([int(n) for n in row.dropna() if pd.notna(n) and str(n).isdigit() and 1 <= int(n) <= max_num])
        except ValueError: logging.warning(f"Ligne avec numéros non convertibles (correlation): {row.tolist()}"); continue

    num_draws = len(draws_list_int)
    if num_draws <= max_lag: print(f"Pas assez de tirages ({num_draws}) pour corrélation (lag max {max_lag})."); return {}

    binary_occurrence_matrix = np.zeros((num_draws, max_num), dtype=int)
    for i, draw_numbers in enumerate(draws_list_int):
        for num_val in draw_numbers:
            if 1 <= num_val <= max_num: binary_occurrence_matrix[i, num_val - 1] = 1

    autocorrelations_results = {num: {} for num in range(1, max_num + 1)}
    if RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS: print("Calcul de l'autocorrélation (affichage partiel):")
    printed_examples_count = 0

    for num_to_analyze in range(1, max_num + 1):
        time_series_for_num = pd.Series(binary_occurrence_matrix[:, num_to_analyze - 1])
        if time_series_for_num.nunique() > 1 and len(time_series_for_num) > max_lag:
            for lag_val in range(1, max_lag + 1):
                if time_series_for_num.var() != 0:
                    try:
                        correlation_value = time_series_for_num.autocorr(lag=lag_val)
                        if isinstance(correlation_value, float) and not np.isnan(correlation_value): autocorrelations_results[num_to_analyze][lag_val] = correlation_value
                    except Exception as e_autocorr: logging.debug(f"Erreur autocorr N°{num_to_analyze}, lag {lag_val}: {e_autocorr}")

            if autocorrelations_results[num_to_analyze] and printed_examples_count < 5 and RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS:
                formatted_corrs = {lag: f"{corr:.3f}" for lag, corr in list(autocorrelations_results[num_to_analyze].items())[:3]}
                print(f"  Numéro {num_to_analyze}: {formatted_corrs} ..."); printed_examples_count += 1

    if RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS:
        example_nums_for_autocorr = [n for n in [1,6,7,9,13,14,19,21,25,30,31,35,37,42,43,49] if n in autocorrelations_results and autocorrelations_results[n]]
        if example_nums_for_autocorr and num_draws > 1:
            num_examples_plot = len(example_nums_for_autocorr); ncols_plot_ac = min(3, num_examples_plot); nrows_plot_ac = (num_examples_plot + ncols_plot_ac - 1) // ncols_plot_ac
            if nrows_plot_ac > 0:
                fig_ac, axes_ac_array = plt.subplots(nrows_plot_ac, ncols_plot_ac, figsize=(ncols_plot_ac*4, nrows_plot_ac*3.5), squeeze=False)
                axes_ac_flat = axes_ac_array.flatten(); confidence_limit = 1.96 / np.sqrt(num_draws)

                for i, num_plot_ac in enumerate(example_nums_for_autocorr):
                    if i < len(axes_ac_flat) and autocorrelations_results[num_plot_ac]:
                        lags_plot = sorted(autocorrelations_results[num_plot_ac].keys()); corrs_plot = [autocorrelations_results[num_plot_ac][l] for l in lags_plot]
                        current_ax = axes_ac_flat[i]; current_ax.bar(lags_plot, corrs_plot, color='cornflowerblue')
                        current_ax.axhline(0, color='grey', linewidth=0.8); current_ax.axhline(confidence_limit, color='red', linestyle='--', linewidth=0.8, label='Limite Signif. (95%)'); current_ax.axhline(-confidence_limit, color='red', linestyle='--', linewidth=0.8)
                        current_ax.set_title(f"Autocorrélation N°{num_plot_ac}"); current_ax.set_xlabel("Lag"); current_ax.set_ylabel("Coeff. Autocorr.")
                        current_ax.set_xticks(lags_plot);
                        if i == 0: current_ax.legend(fontsize='small')
                        current_ax.grid(axis='y', linestyle='--')
                plt.tight_layout(); os.makedirs("plots", exist_ok=True); plt.savefig("plots/autocorrelation_by_number_examples.png"); plt.close();
                print("Graphique exemples autocorrélation sauvegardé.")
    return autocorrelations_results

def analyser_forme_des_tirages(lotto_data, save_prefix="current_dataset_shape"):
    if not RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS: return
    print("\n" + "="*40 + "\n ANALYSE DE LA FORME DES TIRAGES \n" + "="*40)
    if lotto_data.empty: print("Aucune donnée pour analyse de forme."); return

    num_cols = [f"N{i}" for i in range(1, 6)]
    if not all(c in lotto_data.columns for c in num_cols): print("Colonnes N1-N5 manquantes."); return

    try: lotto_numbers_df = lotto_data[num_cols].dropna().astype(int)
    except ValueError: logging.error("Erreur conversion int (analyser_forme)."); print("Erreur conversion (forme)."); return

    if RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS: print("\n--- Analyse Pairs/Impairs ---")
    pairs_counts = lotto_numbers_df.apply(lambda row: sum(n % 2 == 0 for n in row), axis=1); impairs_counts = 5 - pairs_counts
    pair_impair_dist = pd.DataFrame({'pairs': pairs_counts, 'impairs': impairs_counts}).value_counts().sort_index()
    if RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS: print(f"Distribution Pairs/Impairs:\n{pair_impair_dist}")
    if not pair_impair_dist.empty and RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS:
        pair_impair_dist.plot(kind='bar', figsize=(10, 6)); plt.title("Distribution Pairs/Impairs"); plt.xlabel("Combinaison (Pairs, Impairs)"); plt.ylabel("Nb Tirages")
        plt.xticks(rotation=45); plt.grid(axis='y', linestyle='--'); plt.tight_layout(); plt.savefig(f"plots/{save_prefix}_pairs_impairs.png"); plt.close(); print(f"Graphique Pairs/Impairs sauvegardé.")

    if RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS: print("\n--- Analyse Somme Numéros Principaux ---")
    sommes_tirages = lotto_numbers_df.sum(axis=1)
    if RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS: print(f"Stats sommes:\n{sommes_tirages.describe().to_string()}")
    if RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS:
        plt.figure(figsize=(10, 6)); sns.histplot(sommes_tirages, kde=True, bins=30); plt.title("Distribution Somme Numéros Principaux"); plt.xlabel("Somme 5 Numéros"); plt.ylabel("Fréquence")
        plt.grid(axis='y', linestyle='--'); plt.tight_layout(); plt.savefig(f"plots/{save_prefix}_sommes_tirages.png"); plt.close(); print(f"Graphique sommes sauvegardé.")

    if RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS: print("\n--- Répartition Numéros par Dizaines ---")
    dizaines_bins = [0, 9, 19, 29, 39, 49]; dizaines_labels = ['1-9', '10-19', '20-29', '30-39', '40-49']
    repartition_dizaines_counts = pd.DataFrame(columns=dizaines_labels, dtype=int)
    for _, row in lotto_numbers_df.iterrows():
        counts_par_dizaine_row = pd.cut(row, bins=dizaines_bins, labels=dizaines_labels, right=True, include_lowest=True).value_counts().reindex(dizaines_labels, fill_value=0)
        repartition_dizaines_counts = pd.concat([repartition_dizaines_counts, counts_par_dizaine_row.to_frame().T.astype(int)], ignore_index=True)
    if RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS: print(f"Nombre moyen de numéros par dizaine:\n{repartition_dizaines_counts.mean()}")
    if RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS:
        plt.figure(figsize=(12, 7)); sns.boxplot(data=repartition_dizaines_counts); plt.title("Distribution Nb Numéros par Dizaine"); plt.xlabel("Dizaine"); plt.ylabel("Nb Numéros Dizaine")
        plt.grid(axis='y', linestyle='--'); plt.tight_layout(); plt.savefig(f"plots/{save_prefix}_repartition_dizaines.png"); plt.close(); print(f"Graphique répartitions dizaines sauvegardé.")

    if RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS: print("\n--- Analyse Séquences Numéros ---")
    sequences_counts = {2:0, 3:0, 4:0, 5:0}
    for _, row in lotto_numbers_df.iterrows():
        sorted_numbers = sorted(list(row)); found_max_seq_in_row = 0
        for i in range(len(sorted_numbers)):
            current_seq_len = 1
            for j in range(i + 1, len(sorted_numbers)):
                if sorted_numbers[j] - sorted_numbers[j-1] == 1: current_seq_len += 1
                else: break
            if current_seq_len >= 2: found_max_seq_in_row = max(found_max_seq_in_row, current_seq_len)
        if found_max_seq_in_row >= 2: sequences_counts[found_max_seq_in_row] +=1

    if RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS:
        print("Nb tirages avec séquence maximale de longueur X:")
        for length, count in sequences_counts.items():
            if count > 0 : print(f"  Séquences max longueur {length}: {count} tirages")
    if any(sequences_counts.values()) and RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS:
        seq_series = pd.Series(sequences_counts); seq_series_plot = seq_series[seq_series > 0]
        if not seq_series_plot.empty:
            seq_series_plot.plot(kind='bar', figsize=(8,5)); plt.title("Nb Tirages Séquences Numériques (longueur max)"); plt.xlabel("Longueur Max Séquence"); plt.ylabel("Nb Tirages")
            plt.xticks(rotation=0); plt.grid(axis='y', linestyle='--'); plt.tight_layout(); plt.savefig(f"plots/{save_prefix}_sequences_max_par_tirage.png"); plt.close(); print(f"Graphique séquences sauvegardé.")


def analyser_numeros_compagnons(lotto_data, top_n_compagnons=5, save_prefix="current_dataset_companions"):
    if not RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS: return
    print("\n" + "="*40 + "\n ANALYSE DES NUMÉROS COMPAGNONS \n" + "="*40)
    if lotto_data.empty: print("Aucune donnée pour analyse compagnons."); return

    num_cols = [f"N{i}" for i in range(1, 6)]
    if not all(c in lotto_data.columns for c in num_cols): print("Colonnes N1-N5 manquantes."); return

    try: lotto_numbers_df = lotto_data[num_cols].dropna().astype(int)
    except ValueError: logging.error("Erreur conversion int (analyser_compagnons)."); print("Erreur conversion (compagnons)."); return

    compagnons_counts = Counter()
    for _, row in lotto_numbers_df.iterrows():
        for pair in combinations(sorted(list(row)), 2): compagnons_counts[pair] += 1

    if not compagnons_counts: print("Aucune paire de compagnons trouvée."); return

    if RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS:
        print(f"Top {top_n_compagnons * 5} paires de numéros sortant le plus ensemble:")
        for i, (pair, count) in enumerate(compagnons_counts.most_common(top_n_compagnons * 5)):
            print(f"  Paire {pair}: {count} fois");
            if i >= (top_n_compagnons * 5 -1) : break

    num_compagnons_details = {num: Counter() for num in range(1, 50)}
    for pair, count in compagnons_counts.items():
        num_compagnons_details[pair[0]][pair[1]] += count; num_compagnons_details[pair[1]][pair[0]] += count

    if RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS:
        print(f"\nTop {top_n_compagnons} compagnons pour VOS numéros joués ({MES_NUMEROS_PRINCIPAUX_JOUES}):")
        for num_principal in sorted(list(MES_NUMEROS_PRINCIPAUX_JOUES)):
            if num_principal in num_compagnons_details and num_compagnons_details[num_principal]:
                top_comp = num_compagnons_details[num_principal].most_common(top_n_compagnons)
                print(f"  Pour votre numéro {num_principal}: {top_comp}")
            else: print(f"  Pas de données compagnons pour N°{num_principal}.")

def run_full_statistical_analysis(lotto_data_asc):
    if lotto_data_asc is None or lotto_data_asc.empty:
        print("Pas de données pour l'analyse statistique complète.")
        return {}
    print(f"\nLancement de l'analyse statistique (détaillée: {RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS}) sur {len(lotto_data_asc)} tirages.")
    analysis_results = {}
    analysis_results['frequencies'] = analyze_frequencies(lotto_data_asc.copy(), title="Fréquences (Dataset Actuel)", save_prefix="current_dataset_freq")
    if RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS:
        analyze_frequencies_by_day(lotto_data_asc.copy())
        analysis_results['pairs'] = analyze_combinations(lotto_data_asc.copy(), n_numbers=2, top_n=20, save_prefix="pairs_current_dataset")
        analysis_results['triplets'] = analyze_combinations(lotto_data_asc.copy(), n_numbers=3, top_n=10, save_prefix="triplets_current_dataset")
        analysis_results['gaps'] = analyze_gaps(lotto_data_asc.copy())
        analysis_results['temporal_correlation'] = analyze_temporal_correlation(lotto_data_asc.copy(), max_lag=10)
        analyser_forme_des_tirages(lotto_data_asc.copy(), save_prefix="current_dataset_shape")
        analyser_numeros_compagnons(lotto_data_asc.copy(), save_prefix="current_dataset_companions")
    else:
        print("Analyses statistiques détaillées et plots désactivés (RUN_FULL_STAT_ANALYSIS_VERBOSE_PLOTS=False).")
    print("\nAnalyse statistique terminée.")
    return analysis_results

def calculer_statistiques_avancees(historique_df_asc):
    stats = {}
    nombre_total_tirages = len(historique_df_asc)
    if nombre_total_tirages == 0: return stats

    max_num_principal, max_num_comp = 49, 10

    for num in range(1, max_num_principal + 1):
        stats[num] = {'type': 'principal', 'valeur_reelle': num}
        try:
            apparitions = (historique_df_asc[['N1','N2','N3','N4','N5']].apply(pd.to_numeric, errors='coerce') == num).any(axis=1)
            indices_apparitions = historique_df_asc[apparitions].index.tolist()
        except Exception as e: logging.error(f"Erreur calcul apparitions N°P {num}: {e}"); indices_apparitions = []

        stats[num]['frequence_absolue'] = len(indices_apparitions)
        stats[num]['frequence_relative'] = len(indices_apparitions) / nombre_total_tirages if nombre_total_tirages > 0 else 0
        if not indices_apparitions: stats[num].update({'dernier_tirage_idx': -1, 'ecart_actuel': nombre_total_tirages, 'ecarts_observes': [], 'ecart_moyen_specifique': float('inf') if nombre_total_tirages > 0 else 0})
        else:
            stats[num]['dernier_tirage_idx'] = max(indices_apparitions)
            stats[num]['ecart_actuel'] = (nombre_total_tirages - 1) - max(indices_apparitions)
            ecarts = [indices_apparitions[j] - indices_apparitions[j-1] for j in range(1, len(indices_apparitions))] if len(indices_apparitions) > 1 else []
            stats[num]['ecarts_observes'] = ecarts
            stats[num]['ecart_moyen_specifique'] = np.mean(ecarts) if ecarts else float('inf')

    if 'Complementaire' in historique_df_asc.columns:
        for num_c in range(1, max_num_comp + 1):
            cle_stat = f"C{num_c}"; stats[cle_stat] = {'type':'complementaire', 'valeur_reelle': num_c}
            try:
                apparitions_c = (pd.to_numeric(historique_df_asc['Complementaire'], errors='coerce') == num_c)
                indices_apparitions_c = historique_df_asc[apparitions_c].index.tolist()
            except Exception as e: logging.error(f"Erreur calcul apparitions N°C {num_c}: {e}"); indices_apparitions_c = []

            stats[cle_stat]['frequence_absolue'] = len(indices_apparitions_c)
            stats[cle_stat]['frequence_relative'] = len(indices_apparitions_c) / nombre_total_tirages if nombre_total_tirages > 0 else 0
            if not indices_apparitions_c: stats[cle_stat].update({'dernier_tirage_idx': -1, 'ecart_actuel': nombre_total_tirages, 'ecarts_observes': [], 'ecart_moyen_specifique': float('inf') if nombre_total_tirages > 0 else 0})
            else:
                stats[cle_stat]['dernier_tirage_idx'] = max(indices_apparitions_c)
                stats[cle_stat]['ecart_actuel'] = (nombre_total_tirages - 1) - max(indices_apparitions_c)
                ecarts_c = [indices_apparitions_c[j] - indices_apparitions_c[j-1] for j in range(1, len(indices_apparitions_c))] if len(indices_apparitions_c) > 1 else []
                stats[cle_stat]['ecarts_observes'] = ecarts_c
                stats[cle_stat]['ecart_moyen_specifique'] = np.mean(ecarts_c) if ecarts_c else float('inf')
    else: logging.warning("Colonne 'Complementaire' non trouvée pour calculer_statistiques_avancees.")
    return stats

def calculer_score_prediction_custom(num_key, num_stat_dict, nombre_total_tirages_hist, poids, moyenne_globale_ecart_ref_principaux=8.8, moyenne_globale_ecart_ref_comp=9.0):
    score = 0.0
    if num_key not in num_stat_dict or not num_stat_dict[num_key]:
        logging.warning(f"Clé {num_key} non trouvée ou stats vides dans num_stat_dict pour calculer_score_prediction_custom.")
        return 0.0

    num_stats = num_stat_dict[num_key]
    required_keys = ['frequence_absolue', 'frequence_relative', 'ecart_actuel', 'ecart_moyen_specifique', 'type']
    if not all(key in num_stats for key in required_keys):
        logging.warning(f"Statistiques manquantes pour la clé {num_key}: {num_stats}")
        return 0.0

    if num_stats['frequence_absolue'] > 0: score += num_stats['frequence_relative'] * poids['frequence']

    ecart_actuel = num_stats['ecart_actuel']
    ecart_moyen_specifique = num_stats['ecart_moyen_specifique']
    moyenne_globale_ecart_ref = moyenne_globale_ecart_ref_principaux if num_stats['type'] == 'principal' else moyenne_globale_ecart_ref_comp

    if ecart_moyen_specifique != float('inf') and ecart_moyen_specifique > 0:
        proximite_specifique = 1.0 - (abs(ecart_actuel - ecart_moyen_specifique) / ecart_moyen_specifique)
        score += max(0, proximite_specifique) * poids['proximite_ecart_moyen_spec']
        if ecart_actuel > ecart_moyen_specifique: score += ((ecart_actuel / ecart_moyen_specifique) - 1.0) * poids['depassement_ecart_moyen_spec']

    if moyenne_globale_ecart_ref > 0 :
        proximite_globale = 1.0 - (abs(ecart_actuel - moyenne_globale_ecart_ref) / moyenne_globale_ecart_ref)
        score += max(0, proximite_globale) * poids['proximite_ecart_global_ref']
        if ecart_actuel > moyenne_globale_ecart_ref: score += ((ecart_actuel / moyenne_globale_ecart_ref) - 1.0) * poids['depassement_ecart_global_ref']

    if num_stats['frequence_absolue'] == 0: score += poids.get('bonus_jamais_sorti', 0.1)
    elif nombre_total_tirages_hist > 0 and num_stats['frequence_absolue'] < (nombre_total_tirages_hist * 0.01):
        score += poids.get('bonus_rare', 0.05)
    return score

def evaluer_performance_poids(poids, historique_complet_df_asc, nombre_tirages_backtest, min_hist_req):
    if len(historique_complet_df_asc) < (nombre_tirages_backtest + min_hist_req):
        logging.warning(f"Pas assez de données ({len(historique_complet_df_asc)}) pour backtest ({nombre_tirages_backtest + min_hist_req} requis).")
        return -float('inf')

    total_bons_numeros_principaux_trouves = 0
    index_debut_periode_test = len(historique_complet_df_asc) - nombre_tirages_backtest

    if index_debut_periode_test < min_hist_req:
        logging.warning(f"Le premier historique pour backtest ({index_debut_periode_test} tirages) est < min_hist_req ({min_hist_req}).")
        return -float('inf')

    for i in range(index_debut_periode_test, len(historique_complet_df_asc)):
        donnees_pour_prediction = historique_complet_df_asc.iloc[:i]
        tirage_reel_a_predire = historique_complet_df_asc.iloc[i]

        if len(donnees_pour_prediction) < min_hist_req:
            logging.debug(f"Skipping backtest iteration {i}, not enough history: {len(donnees_pour_prediction)} < {min_hist_req}")
            continue

        stats_temp_backtest = calculer_statistiques_avancees(donnees_pour_prediction.copy())
        nombre_tirages_hist_temp = len(donnees_pour_prediction)
        scores_principaux_backtest = []
        for n_p in range(1, 50):
            score_n_p = calculer_score_prediction_custom(n_p, stats_temp_backtest, nombre_tirages_hist_temp, poids)
            scores_principaux_backtest.append({'numero': n_p, 'score': score_n_p})
        scores_principaux_backtest.sort(key=lambda x: x['score'], reverse=True)
        prediction_numeros_principaux = sorted([item['numero'] for item in scores_principaux_backtest[:5]])

        try:
            numeros_reels_principaux = sorted([int(tirage_reel_a_predire[f'N{k}']) for k in range(1,6)])
        except ValueError:
            logging.error(f"Erreur de conversion des numéros réels en int pour le backtest à l'index {i}: {tirage_reel_a_predire}")
            continue

        bons_numeros_ce_tirage = len(set(prediction_numeros_principaux) & set(numeros_reels_principaux))
        total_bons_numeros_principaux_trouves += bons_numeros_ce_tirage
    return total_bons_numeros_principaux_trouves

def optimiser_poids_par_grille(historique_complet_df_asc, nombre_tirages_backtest=49, min_hist_req=56):
    print("\n" + "="*40 + "\n OPTIMISATION DES POIDS (RECHERCHE PAR GRILLE) \n" + "="*40)
    grille_valeurs_poids = {
        'frequence': [0.1, 0.14],
        'proximite_ecart_moyen_spec': [0.4, 0.7],
        'depassement_ecart_moyen_spec': [0.2, 0.3],
        'proximite_ecart_global_ref': [0.1, 0.14],
        'depassement_ecart_global_ref': [0.02, 0.07],
        'bonus_jamais_sorti': [0.07, 0.1],
        'bonus_rare': [0.02, 0.07]
    }
    cles_des_poids = list(grille_valeurs_poids.keys())
    valeurs_combinaisons_poids = [grille_valeurs_poids[k] for k in cles_des_poids]
    meilleurs_poids_trouves = None
    meilleure_performance_obtenue = -1

    combinaisons_de_poids_a_tester = list(product(*valeurs_combinaisons_poids))
    nombre_total_combinaisons = len(combinaisons_de_poids_a_tester)

    if nombre_total_combinaisons > MAX_GRID_SEARCH_CUSTOM_COMBINATIONS:
        print(f"Attention: Limitant le test à {MAX_GRID_SEARCH_CUSTOM_COMBINATIONS} combinaisons de poids sur {nombre_total_combinaisons}.")
        combinaisons_de_poids_a_tester = combinaisons_de_poids_a_tester[:MAX_GRID_SEARCH_CUSTOM_COMBINATIONS]
        nombre_total_combinaisons = MAX_GRID_SEARCH_CUSTOM_COMBINATIONS


    print(f"Début du test de {nombre_total_combinaisons} combinaisons de poids...")
    if nombre_total_combinaisons == 0:
        print("Aucune combinaison de poids à tester."); return {'frequence':0.1, 'proximite_ecart_moyen_spec':0.5, 'depassement_ecart_moyen_spec':0.3, 'proximite_ecart_global_ref':0.1, 'depassement_ecart_global_ref':0.05, 'bonus_jamais_sorti':0.1, 'bonus_rare':0.05}

    for i, valeurs_poids_actuels in enumerate(combinaisons_de_poids_a_tester):
        poids_actuels_dict = dict(zip(cles_des_poids, valeurs_poids_actuels))
        if (i + 1) % 10 == 0 or i == 0 or (i + 1) == nombre_total_combinaisons :
            print(f"  Test de la combinaison de poids {i+1}/{nombre_total_combinaisons}...")

        performance_actuelle = evaluer_performance_poids(
            poids_actuels_dict, historique_complet_df_asc,
            nombre_tirages_backtest, min_hist_req
        )
        if performance_actuelle > meilleure_performance_obtenue:
            meilleure_performance_obtenue = performance_actuelle
            meilleurs_poids_trouves = poids_actuels_dict
            print(f"    NOUVEAUX MEILLEURS POIDS TROUVÉS ! Combinaison {i+1}. Perf: {meilleure_performance_obtenue}.")
            if meilleurs_poids_trouves:
                try:
                    with open("meilleurs_poids_intermediaires.json", "w") as f_json:
                        json.dump({'poids': meilleurs_poids_trouves, 'performance': float(meilleure_performance_obtenue)}, f_json)
                except Exception as e_save: print(f"Erreur sauvegarde intermédiaire: {e_save}")

    if meilleurs_poids_trouves:
        print(f"Meilleurs poids trouvés : {meilleurs_poids_trouves}")
        print(f"Meilleure performance : {meilleure_performance_obtenue}")
    else:
        print("Aucune combinaison de poids n'a amélioré la performance. Poids par défaut.")
        meilleurs_poids_trouves = {'frequence':0.1, 'proximite_ecart_moyen_spec':0.5, 'depassement_ecart_moyen_spec':0.3, 'proximite_ecart_global_ref':0.1, 'depassement_ecart_global_ref':0.05, 'bonus_jamais_sorti':0.1, 'bonus_rare':0.05}
    print("="*60)
    return meilleurs_poids_trouves

def preparer_donnees_pour_ml_numero(historique_df_asc, num_cible, type_cible='principal'):
    X_data, y_data = [], []
    min_historique_pour_features = 21
    if len(historique_df_asc) < min_historique_pour_features:
        logging.warning(f"Historique trop court ({len(historique_df_asc)}) pour préparer les données ML pour {num_cible}.")
        return np.array(X_data), np.array(y_data)

    for i in range(min_historique_pour_features, len(historique_df_asc)):
        hist_avant_tirage_i_dans_fenetre = historique_df_asc.iloc[:i]
        tirage_actuel_i = historique_df_asc.iloc[i]

        stats_avant_i = calculer_statistiques_avancees(hist_avant_tirage_i_dans_fenetre.copy())
        cle_stat_cible = num_cible if type_cible == 'principal' else f"C{num_cible}"

        if cle_stat_cible not in stats_avant_i or not stats_avant_i[cle_stat_cible]:
            logging.debug(f"Clé {cle_stat_cible} non trouvée ou stats vides dans stats_avant_i pour tirage {i} (index df) dans la préparation ML.")
            continue
        stat_num_cible = stats_avant_i[cle_stat_cible]
        sortie_tirage_precedent, apparitions_3_derniers, apparitions_5_derniers = 0, 0, 0

        if not hist_avant_tirage_i_dans_fenetre.empty:
            dernier_tirage_dans_hist = hist_avant_tirage_i_dans_fenetre.iloc[-1]
            try:
                if type_cible == 'principal':
                    if num_cible in {int(dernier_tirage_dans_hist[f'N{k}']) for k in range(1,6) if pd.notna(dernier_tirage_dans_hist[f'N{k}'])}:
                        sortie_tirage_precedent = 1
                else:
                    if pd.notna(dernier_tirage_dans_hist['Complementaire']) and int(dernier_tirage_dans_hist['Complementaire']) == num_cible:
                        sortie_tirage_precedent = 1
            except ValueError: logging.warning(f"Erreur conversion int 'sortie_tirage_precedent' (ML) N°{num_cible}.")

            hist_3_derniers = hist_avant_tirage_i_dans_fenetre.tail(3)
            for _, row_3 in hist_3_derniers.iterrows():
                try:
                    if type_cible == 'principal':
                        if num_cible in {int(row_3[f'N{k}']) for k in range(1,6) if pd.notna(row_3[f'N{k}'])}: apparitions_3_derniers += 1
                    else:
                        if pd.notna(row_3['Complementaire']) and int(row_3['Complementaire']) == num_cible: apparitions_3_derniers += 1
                except ValueError: logging.warning(f"Erreur conversion int 'apparitions_3_derniers' (ML) N°{num_cible}.")

            hist_5_derniers = hist_avant_tirage_i_dans_fenetre.tail(5)
            for _, row_5 in hist_5_derniers.iterrows():
                try:
                    if type_cible == 'principal':
                        if num_cible in {int(row_5[f'N{k}']) for k in range(1,6) if pd.notna(row_5[f'N{k}'])}: apparitions_5_derniers += 1
                    else:
                        if pd.notna(row_5['Complementaire']) and int(row_5['Complementaire']) == num_cible: apparitions_5_derniers += 1
                except ValueError: logging.warning(f"Erreur conversion int 'apparitions_5_derniers' (ML) N°{num_cible}.")

        ecart_moyen_spec_feature = stat_num_cible['ecart_moyen_specifique']
        if ecart_moyen_spec_feature == float('inf'):
            ecart_moyen_spec_feature = len(hist_avant_tirage_i_dans_fenetre) if stat_num_cible['frequence_absolue'] <=1 else -1

        features_pour_exemple = [
            stat_num_cible.get('frequence_relative', 0.0),
            stat_num_cible.get('ecart_actuel', len(hist_avant_tirage_i_dans_fenetre)),
            ecart_moyen_spec_feature,
            len(stat_num_cible.get('ecarts_observes', [])),
            sortie_tirage_precedent, apparitions_3_derniers, apparitions_5_derniers
        ]
        current_X_len = len(X_data)
        X_data.append(features_pour_exemple)

        try:
            if type_cible == 'principal':
                numeros_gagnants_principaux_i = {int(tirage_actuel_i[f'N{k}']) for k in range(1,6) if pd.notna(tirage_actuel_i[f'N{k}'])}
                y_data.append(1 if num_cible in numeros_gagnants_principaux_i else 0)
            else:
                if pd.notna(tirage_actuel_i['Complementaire']):
                    y_data.append(1 if int(tirage_actuel_i['Complementaire']) == num_cible else 0)
                else: y_data.append(0)
        except (ValueError, TypeError) as e_target:
            logging.error(f"Erreur conversion cible y N°{num_cible} tirage index {i}: {e_target}. Ligne: {tirage_actuel_i}")
            if len(X_data) > current_X_len : X_data.pop()
            continue
    return np.array(X_data), np.array(y_data)

def _calculer_features_recence_pour_prediction(donnees_historiques_fenetre, num_cible_pred, type_cible_pred):
    sortie_tirage_precedent_pred, apparitions_3_derniers_pred, apparitions_5_derniers_pred = 0, 0, 0
    if not donnees_historiques_fenetre.empty:
        dernier_tirage_hist_pred = donnees_historiques_fenetre.iloc[-1]
        try:
            if type_cible_pred == 'principal':
                if all(f'N{k}' in dernier_tirage_hist_pred for k in range(1,6)):
                    if num_cible_pred in {int(dernier_tirage_hist_pred[f'N{k}']) for k in range(1,6) if pd.notna(dernier_tirage_hist_pred[f'N{k}'])}:
                        sortie_tirage_precedent_pred = 1
            else:
                if 'Complementaire' in dernier_tirage_hist_pred and pd.notna(dernier_tirage_hist_pred['Complementaire']):
                    if int(dernier_tirage_hist_pred['Complementaire']) == num_cible_pred: sortie_tirage_precedent_pred = 1
        except (ValueError, TypeError) as e_pred_rec: logging.warning(f"Erreur conversion _calculer_features_recence (dernier) N°{num_cible_pred}: {e_pred_rec}")

        hist_3_derniers_pred = donnees_historiques_fenetre.tail(3)
        for _, row_3_pred in hist_3_derniers_pred.iterrows():
            try:
                if type_cible_pred == 'principal':
                    if all(f'N{k}' in row_3_pred for k in range(1,6)):
                        if num_cible_pred in {int(row_3_pred[f'N{k}']) for k in range(1,6) if pd.notna(row_3_pred[f'N{k}'])}: apparitions_3_derniers_pred += 1
                else:
                    if 'Complementaire' in row_3_pred and pd.notna(row_3_pred['Complementaire']):
                        if int(row_3_pred['Complementaire']) == num_cible_pred: apparitions_3_derniers_pred += 1
            except (ValueError, TypeError) as e_pred_rec3: logging.warning(f"Erreur conversion _calculer_features_recence (3 derniers) N°{num_cible_pred}: {e_pred_rec3}")

        hist_5_derniers_pred = donnees_historiques_fenetre.tail(5)
        for _, row_5_pred in hist_5_derniers_pred.iterrows():
            try:
                if type_cible_pred == 'principal':
                    if all(f'N{k}' in row_5_pred for k in range(1,6)):
                        if num_cible_pred in {int(row_5_pred[f'N{k}']) for k in range(1,6) if pd.notna(row_5_pred[f'N{k}'])}: apparitions_5_derniers_pred += 1
                else:
                    if pd.notna(row_5_pred['Complementaire']) and int(row_5_pred['Complementaire']) == num_cible_pred: apparitions_5_derniers_pred += 1
            except (ValueError, TypeError) as e_pred_rec5: logging.warning(f"Erreur conversion _calculer_features_recence (5 derniers) N°{num_cible_pred}: {e_pred_rec5}")
    return sortie_tirage_precedent_pred, apparitions_3_derniers_pred, apparitions_5_derniers_pred

def entrainer_et_predire_ml_pour_backtest(historique_complet_asc_total, index_tirage_a_predire, modeles_principaux, modeles_complementaires, taille_fenetre_entrainement_ml=98):
    debut_fenetre = max(0, index_tirage_a_predire - taille_fenetre_entrainement_ml)
    donnees_entrainement_ml_fenetre = historique_complet_asc_total.iloc[debut_fenetre:index_tirage_a_predire]
    min_hist_pour_features_ml = 21

    if len(donnees_entrainement_ml_fenetre) < min_hist_pour_features_ml :
        logging.warning(f"Fenêtre d'entraînement ML trop petite ({len(donnees_entrainement_ml_fenetre)}, min {min_hist_pour_features_ml}) pour tirage index {index_tirage_a_predire}. Prédictions ML par défaut.")
        pred_p_ml_default = sorted(list(MES_NUMEROS_PRINCIPAUX_JOUES))[:5] if MES_NUMEROS_PRINCIPAUX_JOUES else random.sample(range(1,50),5)
        pred_c_ml_default = list(MES_COMPLEMENTAIRES_POSSIBLES_JOUES)[0] if MES_COMPLEMENTAIRES_POSSIBLES_JOUES else random.randint(1,10)
        return pred_p_ml_default, pred_c_ml_default

    print(f"      Utilisation d'une fenêtre de {len(donnees_entrainement_ml_fenetre)} tirages (indices {debut_fenetre} à {index_tirage_a_predire-1}) pour l'entraînement ML du tirage à l'index {index_tirage_a_predire}.")

    stats_pour_creation_features_prediction = calculer_statistiques_avancees(donnees_entrainement_ml_fenetre.copy())

    param_grid_rf = {'n_estimators': [50, 100], 'max_depth': [None, 10, 20], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 3, 5]}
    default_n_estimators = 100

    predictions_probabilites_principaux = []
    for num_p in range(1, 50):
        X_train_num_p, y_train_num_p = preparer_donnees_pour_ml_numero(donnees_entrainement_ml_fenetre.copy(), num_p, 'principal')

        if len(X_train_num_p) < 10 or len(np.unique(y_train_num_p)) < 2:
            modeles_principaux[num_p] = None
            predictions_probabilites_principaux.append({'numero': num_p, 'proba': 0.0})
            continue

        current_model_p = None
        if USE_GRID_SEARCH_FOR_FULL_ANALYSIS_ML:
            cv_splits = min(3, len(X_train_num_p) // (sum(param_grid_rf.get('min_samples_split', [2])) // len(param_grid_rf.get('min_samples_split', [2]))) if len(X_train_num_p) > 20 else 2)
            cv_splits = max(2, cv_splits)
            grid_search_p = GridSearchCV(RandomForestClassifier(random_state=42, class_weight='balanced'), param_grid_rf, cv=cv_splits, scoring='roc_auc', n_jobs=-1, error_score='raise')
            try:
                grid_search_p.fit(X_train_num_p, y_train_num_p)
                current_model_p = grid_search_p.best_estimator_
            except ValueError as e_gs_p:
                logging.warning(f"Erreur GridSearchCV Principal N°{num_p} (Backtest): {e_gs_p}. Modèle par défaut.")
                current_model_p = RandomForestClassifier(n_estimators=default_n_estimators, random_state=42, class_weight='balanced', n_jobs=-1)
                current_model_p.fit(X_train_num_p, y_train_num_p)
        else:
            current_model_p = RandomForestClassifier(n_estimators=default_n_estimators, random_state=42, class_weight='balanced', n_jobs=-1)
            current_model_p.fit(X_train_num_p, y_train_num_p)
        modeles_principaux[num_p] = current_model_p

        if num_p not in stats_pour_creation_features_prediction:
            logging.warning(f"Stats non trouvées N°P {num_p} création features prédiction (backtest).")
            predictions_probabilites_principaux.append({'numero': num_p, 'proba': 0.0})
            continue
        stat_num_cible_p = stats_pour_creation_features_prediction[num_p]
        s_t_p_pred, a_3_d_pred, a_5_d_pred = _calculer_features_recence_pour_prediction(donnees_entrainement_ml_fenetre, num_p, 'principal')
        ecart_moyen_spec_feature_pred_p = stat_num_cible_p.get('ecart_moyen_specifique', float('inf'))
        if ecart_moyen_spec_feature_pred_p == float('inf'):
            ecart_moyen_spec_feature_pred_p = len(donnees_entrainement_ml_fenetre) if stat_num_cible_p.get('frequence_absolue',0) <=1 else -1
        features_pour_prediction_p = np.array([[
            stat_num_cible_p.get('frequence_relative', 0.0),
            stat_num_cible_p.get('ecart_actuel', len(donnees_entrainement_ml_fenetre)),
            ecart_moyen_spec_feature_pred_p,
            len(stat_num_cible_p.get('ecarts_observes', [])),
            s_t_p_pred, a_3_d_pred, a_5_d_pred
        ]])
        if modeles_principaux[num_p]:
            try:
                proba_sortie_p = modeles_principaux[num_p].predict_proba(features_pour_prediction_p)[0][1]
                predictions_probabilites_principaux.append({'numero': num_p, 'proba': proba_sortie_p})
            except Exception as e_predict_p:
                logging.error(f"Erreur predict_proba Principal N°{num_p} (Backtest): {e_predict_p}")
                predictions_probabilites_principaux.append({'numero': num_p, 'proba': 0.0})
        else: predictions_probabilites_principaux.append({'numero': num_p, 'proba': 0.0})


    predictions_probabilites_complementaires = []
    for num_c in range(1, 11):
        X_train_num_c, y_train_num_c = preparer_donnees_pour_ml_numero(donnees_entrainement_ml_fenetre.copy(), num_c, 'complementaire')
        cle_stat_c_pred = f"C{num_c}"

        if len(X_train_num_c) < 10 or len(np.unique(y_train_num_c)) < 2:
            modeles_complementaires[num_c] = None
            predictions_probabilites_complementaires.append({'numero': num_c, 'proba': 0.0})
            continue

        current_model_c = None
        if USE_GRID_SEARCH_FOR_FULL_ANALYSIS_ML:
            cv_splits_c = min(3, len(X_train_num_c) // (sum(param_grid_rf.get('min_samples_split', [2])) // len(param_grid_rf.get('min_samples_split', [2]))) if len(X_train_num_c) > 20 else 2)
            cv_splits_c = max(2, cv_splits_c)
            grid_search_c = GridSearchCV(RandomForestClassifier(random_state=42, class_weight='balanced'), param_grid_rf, cv=cv_splits_c, scoring='roc_auc', n_jobs=-1, error_score='raise')
            try:
                grid_search_c.fit(X_train_num_c, y_train_num_c)
                current_model_c = grid_search_c.best_estimator_
            except ValueError as e_gs_c:
                logging.warning(f"Erreur GridSearchCV Complémentaire N°{num_c} (Backtest): {e_gs_c}. Modèle par défaut.")
                current_model_c = RandomForestClassifier(n_estimators=default_n_estimators, random_state=42, class_weight='balanced', n_jobs=-1)
                current_model_c.fit(X_train_num_c, y_train_num_c)
        else:
            current_model_c = RandomForestClassifier(n_estimators=default_n_estimators, random_state=42, class_weight='balanced', n_jobs=-1)
            current_model_c.fit(X_train_num_c, y_train_num_c)
        modeles_complementaires[num_c] = current_model_c

        if cle_stat_c_pred not in stats_pour_creation_features_prediction:
            logging.warning(f"Stats non trouvées N°C {num_c} création features prédiction (backtest).")
            predictions_probabilites_complementaires.append({'numero': num_c, 'proba': 0.0})
            continue
        stat_num_cible_c = stats_pour_creation_features_prediction[cle_stat_c_pred]
        s_t_p_pred_c, a_3_d_pred_c, a_5_d_pred_c = _calculer_features_recence_pour_prediction(donnees_entrainement_ml_fenetre, num_c, 'complementaire')
        ecart_moyen_spec_feature_pred_c = stat_num_cible_c.get('ecart_moyen_specifique', float('inf'))
        if ecart_moyen_spec_feature_pred_c == float('inf'):
            ecart_moyen_spec_feature_pred_c = len(donnees_entrainement_ml_fenetre) if stat_num_cible_c.get('frequence_absolue',0) <=1 else -1
        features_pour_prediction_c = np.array([[
            stat_num_cible_c.get('frequence_relative', 0.0),
            stat_num_cible_c.get('ecart_actuel', len(donnees_entrainement_ml_fenetre)),
            ecart_moyen_spec_feature_pred_c,
            len(stat_num_cible_c.get('ecarts_observes', [])),
            s_t_p_pred_c, a_3_d_pred_c, a_5_d_pred_c
        ]])
        if modeles_complementaires[num_c]:
            try:
                proba_sortie_c = modeles_complementaires[num_c].predict_proba(features_pour_prediction_c)[0][1]
                predictions_probabilites_complementaires.append({'numero': num_c, 'proba': proba_sortie_c})
            except Exception as e_predict_c:
                logging.error(f"Erreur predict_proba Complémentaire N°{num_c} (Backtest): {e_predict_c}")
                predictions_probabilites_complementaires.append({'numero': num_c, 'proba': 0.0})
        else: predictions_probabilites_complementaires.append({'numero': num_c, 'proba': 0.0})

    predictions_probabilites_principaux.sort(key=lambda x: x['proba'], reverse=True)
    predictions_probabilites_complementaires.sort(key=lambda x: x['proba'], reverse=True)

    prediction_principaux_ml = sorted([item['numero'] for item in predictions_probabilites_principaux[:5]])
    prediction_complementaire_ml = predictions_probabilites_complementaires[0]['numero'] if predictions_probabilites_complementaires and predictions_probabilites_complementaires[0]['proba'] > 0 else None

    return prediction_principaux_ml, prediction_complementaire_ml

# --- Nouvelle fonction pour calculer les "quasiment-gagnants" personnalisés ---
def calculate_custom_almost_wins(predicted_nums, actual_nums, predicted_comp=None, actual_comp=None, max_num_main=49, max_num_comp=10):
    """
    Calcule les métriques de "quasiment-gagnants" basées sur la dizaine, la proximité, la parité, miroir et circulaire.
    """
    results = {
        'num_in_same_decade': 0, 'num_plus_minus_1': 0, 'num_plus_minus_2': 0,
        'num_same_decade_and_parity': 0, 'num_mirror': 0,
        'num_circular_plus_minus_1': 0, 'num_circular_plus_minus_2': 0,
        'comp_in_same_decade': 0, 'comp_plus_minus_1': 0, 'comp_plus_minus_2': 0,
        'comp_same_decade_and_parity': 0, 'comp_mirror': 0,
        'comp_circular_plus_minus_1': 0, 'comp_circular_plus_minus_2': 0
    }

    # --- Helper function for mirror ---
    def get_mirror(n):
        if 10 <= n <= 99: # S'applique aux nombres à deux chiffres
            s = str(n)
            if s[0] == s[1]: return None # Miroir non défini pour 11, 22 etc.
            mirrored = int(s[1] + s[0])
            return mirrored
        return None

    # --- Helper function for circular distance ---
    def circular_distance(n1, n2, max_val):
        diff = abs(n1 - n2)
        return min(diff, max_val - diff)

    # --- Analyse pour les numéros principaux ---
    # Utiliser des sets pour compter les numéros réels uniques qui correspondent aux critères
    # pour éviter de compter plusieurs fois le même numéro réel s'il correspond à plusieurs numéros prédits.
    unique_matches = {key: set() for key in [
        'num_in_same_decade', 'num_plus_minus_1', 'num_plus_minus_2_strict',
        'num_same_decade_and_parity', 'num_mirror',
        'num_circular_plus_minus_1', 'num_circular_plus_minus_2_strict'
    ]}

    for p_num in predicted_nums:
        p_decade_start = (p_num - 1) // 10 * 10
        p_is_even = (p_num % 2 == 0)
        p_mirror = get_mirror(p_num)

        for a_num in actual_nums:
            a_decade_start = (a_num - 1) // 10 * 10
            a_is_even = (a_num % 2 == 0)

            if p_decade_start == a_decade_start:
                unique_matches['num_in_same_decade'].add(a_num)
                if p_is_even == a_is_even:
                    unique_matches['num_same_decade_and_parity'].add(a_num)

            diff_abs = abs(p_num - a_num)
            if diff_abs == 1:
                unique_matches['num_plus_minus_1'].add(a_num)
            elif diff_abs == 2:
                unique_matches['num_plus_minus_2_strict'].add(a_num) # Sera filtré plus tard

            if p_mirror is not None and p_mirror == a_num:
                unique_matches['num_mirror'].add(a_num)

            circ_dist = circular_distance(p_num, a_num, max_num_main)
            if circ_dist == 1:
                unique_matches['num_circular_plus_minus_1'].add(a_num)
            elif circ_dist == 2:
                unique_matches['num_circular_plus_minus_2_strict'].add(a_num) # Sera filtré

    results['num_in_same_decade'] = len(unique_matches['num_in_same_decade'])
    results['num_plus_minus_1'] = len(unique_matches['num_plus_minus_1'])
    # S'assurer que +/-2 n'inclut pas ceux déjà comptés par +/-1
    results['num_plus_minus_2'] = len(unique_matches['num_plus_minus_2_strict'] - unique_matches['num_plus_minus_1'])
    results['num_same_decade_and_parity'] = len(unique_matches['num_same_decade_and_parity'])
    results['num_mirror'] = len(unique_matches['num_mirror'])
    results['num_circular_plus_minus_1'] = len(unique_matches['num_circular_plus_minus_1'])
    results['num_circular_plus_minus_2'] = len(unique_matches['num_circular_plus_minus_2_strict'] - unique_matches['num_circular_plus_minus_1'])


    # --- Analyse pour le numéro complémentaire (si fourni) ---
    if predicted_comp is not None and actual_comp is not None:
        p_comp_decade_start = (predicted_comp - 1) // 10 * 10
        p_comp_is_even = (predicted_comp % 2 == 0)
        p_comp_mirror = get_mirror(predicted_comp)

        a_comp_decade_start = (actual_comp - 1) // 10 * 10
        a_comp_is_even = (actual_comp % 2 == 0)

        if p_comp_decade_start == a_comp_decade_start:
            results['comp_in_same_decade'] = 1
            if p_comp_is_even == a_comp_is_even:
                results['comp_same_decade_and_parity'] = 1

        diff_abs_comp = abs(predicted_comp - actual_comp)
        if diff_abs_comp == 1:
            results['comp_plus_minus_1'] = 1
        elif diff_abs_comp == 2: # Ne sera compté que si ce n'est pas déjà +/-1
            results['comp_plus_minus_2'] = 1

        if p_comp_mirror is not None and p_comp_mirror == actual_comp:
            results['comp_mirror'] = 1

        circ_dist_comp = circular_distance(predicted_comp, actual_comp, max_num_comp)
        if circ_dist_comp == 1:
            results['comp_circular_plus_minus_1'] = 1
        elif circ_dist_comp == 2: # Ne sera compté que si ce n'est pas déjà circ +/-1
             results['comp_circular_plus_minus_2'] = 1

    return results


def tester_performance_predictive_historique(historique_complet_df_asc, nombre_de_tirages_a_tester, poids_optimises):
    print("\n" + "="*40 + "\n BACKTESTING COMPARATIF (Modèle Custom vs Modèle ML) \n" + "="*40)
    min_historique_requis_backtest_global = 63
    taille_fenetre_entrainement_ml_backtest = 105

    if len(historique_complet_df_asc) < (max(min_historique_requis_backtest_global, taille_fenetre_entrainement_ml_backtest) + nombre_de_tirages_a_tester):
        print(f"Pas assez de données pour backtest complet ({len(historique_complet_df_asc)}).")
        print(f"Besoin d'au moins {max(min_historique_requis_backtest_global, taille_fenetre_entrainement_ml_backtest) + nombre_de_tirages_a_tester} tirages.")
        return [], [] # Retourner des listes vides pour les résultats

    # Initialisation des listes pour stocker les résultats détaillés par tirage
    backtest_details_custom = []
    backtest_details_ml = []

    modeles_ml_principaux_backtest = {num: None for num in range(1,50)}
    modeles_ml_complementaires_backtest = {num: None for num in range(1,11)}
    index_debut_backtest_loop = len(historique_complet_df_asc) - nombre_de_tirages_a_tester

    for i in range(index_debut_backtest_loop, len(historique_complet_df_asc)):
        donnees_historiques_pour_prediction_i = historique_complet_df_asc.iloc[:i]
        tirage_reel_i = historique_complet_df_asc.iloc[i]

        if len(donnees_historiques_pour_prediction_i) < min_historique_requis_backtest_global:
            logging.info(f"Skipping backtest tirage index {i}: hist custom insuffisant ({len(donnees_historiques_pour_prediction_i)}).")
            continue

        # --- Modèle Custom ---
        stats_custom_i = calculer_statistiques_avancees(donnees_historiques_pour_prediction_i.copy())
        nb_tirages_hist_i = len(donnees_historiques_pour_prediction_i)
        scores_p_custom_i = [{'numero':n, 'score':calculer_score_prediction_custom(n, stats_custom_i, nb_tirages_hist_i, poids_optimises)} for n in range(1,50)]
        scores_c_custom_i = [{'numero':n, 'score':calculer_score_prediction_custom(f"C{n}", stats_custom_i, nb_tirages_hist_i, poids_optimises)} for n in range(1,11)]
        scores_p_custom_i.sort(key=lambda x: x['score'], reverse=True)
        scores_c_custom_i.sort(key=lambda x: x['score'], reverse=True)
        pred_p_custom_i = sorted([item['numero'] for item in scores_p_custom_i[:5]])
        pred_c_custom_i = scores_c_custom_i[0]['numero'] if scores_c_custom_i and scores_c_custom_i[0]['score'] > 0 else None

        # --- Modèle ML ---
        pred_p_ml_i, pred_c_ml_i = ["N/A"]*5, "N/A" # Default
        if RUN_BACKTESTING_ML_IN_FULL_ANALYSIS:
            if len(donnees_historiques_pour_prediction_i) >= taille_fenetre_entrainement_ml_backtest:
                print("  Début entraînement/prédiction ML pour ce tirage du backtest...")
                pred_p_ml_i, pred_c_ml_i = entrainer_et_predire_ml_pour_backtest(
                    historique_complet_df_asc, i, modeles_ml_principaux_backtest,
                    modeles_ml_complementaires_backtest, taille_fenetre_entrainement_ml=taille_fenetre_entrainement_ml_backtest
                )
                print("  Fin entraînement/prédiction ML pour ce tirage du backtest.")
            else:
                print(f"  Skipping ML pour tirage index {i}: hist ML insuffisant ({len(donnees_historiques_pour_prediction_i)} < {taille_fenetre_entrainement_ml_backtest}).")

        # --- Numéros Réels ---
        try:
            reels_p_i = sorted([int(tirage_reel_i[f'N{k}']) for k in range(1,6)])
            reel_c_i = int(tirage_reel_i['Complementaire']) if pd.notna(tirage_reel_i['Complementaire']) else None
        except (ValueError, TypeError) as e_reel:
            logging.error(f"Erreur conversion numéros réels backtest index {i}: {tirage_reel_i}, {e_reel}")
            continue

        # --- Évaluation Modèle Custom ---
        bons_p_custom_i = len(set(pred_p_custom_i) & set(reels_p_i))
        bon_c_custom_i = 1 if pred_c_custom_i is not None and reel_c_i is not None and pred_c_custom_i == reel_c_i else 0
        aw_custom_results_i = calculate_custom_almost_wins(pred_p_custom_i, reels_p_i, pred_c_custom_i, reel_c_i)

        current_custom_detail = {
            'date': tirage_reel_i['Date'], 'bons_p': bons_p_custom_i, 'bon_c': bon_c_custom_i,
            **{f"aw_{k}": v for k, v in aw_custom_results_i.items()} # Ajoute tous les "almost wins"
        }
        backtest_details_custom.append(current_custom_detail)

        # --- Évaluation Modèle ML ---
        bons_p_ml_i_val = 0
        bon_c_ml_i_val = 0
        aw_ml_results_i = {} # Initialiser vide
        if RUN_BACKTESTING_ML_IN_FULL_ANALYSIS and pred_p_ml_i != ["N/A"]*5 : # S'assurer que la prédiction ML a eu lieu
            bons_p_ml_i_val = len(set(pred_p_ml_i) & set(reels_p_i))
            bon_c_ml_i_val = 1 if pred_c_ml_i is not None and pred_c_ml_i != "N/A" and reel_c_i is not None and pred_c_ml_i == reel_c_i else 0
            aw_ml_results_i = calculate_custom_almost_wins(pred_p_ml_i, reels_p_i, pred_c_ml_i, reel_c_i)

        current_ml_detail = {
            'date': tirage_reel_i['Date'], 'bons_p': bons_p_ml_i_val, 'bon_c': bon_c_ml_i_val,
            **{f"aw_{k}": v for k, v in aw_ml_results_i.items()}
        }
        backtest_details_ml.append(current_ml_detail)


        print(f"  Résultats pour tirage {tirage_reel_i['Date'].strftime('%Y-%m-%d')}:")
        print(f"    Modèle Custom: Prédit P:{pred_p_custom_i} + C:{pred_c_custom_i} -> Trouvé {bons_p_custom_i}/5 N°P, {bon_c_custom_i}/1 N°C")
        if RUN_BACKTESTING_ML_IN_FULL_ANALYSIS:
             print(f"    Modèle ML (RF): Prédit P:{pred_p_ml_i} + C:{pred_c_ml_i} -> Trouvé {bons_p_ml_i_val}/5 N°P, {bon_c_ml_i_val}/1 N°C")
        print(f"    Tirage Réel         : P:{reels_p_i} + C:{reel_c_i}")

    # --- Affichage des résumés de backtest ---
    df_backtest_custom = pd.DataFrame(backtest_details_custom)
    df_backtest_ml = pd.DataFrame(backtest_details_ml)

    for nom_modele, df_results in [("Modèle Custom (Poids Optimisés)", df_backtest_custom),
                                   ("Modèle ML (Random Forest)", df_backtest_ml)]:
        if not RUN_BACKTESTING_ML_IN_FULL_ANALYSIS and nom_modele == "Modèle ML (Random Forest)":
            continue
        if not df_results.empty:
            print(f"\n--- Résumé Détaillé du Backtest pour {nom_modele} sur {len(df_results)} tirages ---")
            print(f"  Nombre total de bons numéros principaux trouvés : {df_results['bons_p'].sum()}")
            print(f"  Nombre total de bons numéros complémentaires trouvés : {df_results['bon_c'].sum()}")
            print(f"  Meilleure prédiction de numéros principaux en un tirage : {df_results['bons_p'].max()}/5")
            for k_bons in range(6):
                count_k = (df_results['bons_p'] == k_bons).sum()
                if count_k > 0:
                    print(f"    Nombre de fois où {k_bons} bons N°P ont été trouvés : {count_k}")

            if RUN_ALMOST_WINNERS_ANALYSIS_IN_BACKTEST:
                print(f"  'Quasiment-Gagnants' (Rangs Loto classiques):")
                # Recalculer les rangs Loto classiques si nécessaire ou les stocker pendant la boucle
                aw_loto_4_sur_5 = (df_results['bons_p'] == 4).sum()
                aw_loto_3_sur_5_et_C = ((df_results['bons_p'] == 3) & (df_results['bon_c'] == 1)).sum()
                aw_loto_2_sur_5_et_C = ((df_results['bons_p'] == 2) & (df_results['bon_c'] == 1)).sum()
                print(f"    4 sur 5 N°P: {aw_loto_4_sur_5} fois")
                print(f"    3 sur 5 N°P + N°C: {aw_loto_3_sur_5_et_C} fois")
                print(f"    2 sur 5 N°P + N°C: {aw_loto_2_sur_5_et_C} fois")

                print(f"\n  'Quasiment-Gagnants' Personnalisés (Détails par tirage - Moyenne sur backtest):")
                aw_cols_num = [col for col in df_results.columns if col.startswith('aw_num_')]
                aw_cols_comp = [col for col in df_results.columns if col.startswith('aw_comp_')]

                print("    Pour les Numéros Principaux:")
                for col in aw_cols_num:
                    col_name_fr = col.replace('aw_num_', '').replace('_', ' ').capitalize()
                    print(f"      {col_name_fr}: Moy. {df_results[col].mean():.2f}, Max {df_results[col].max()}, Total {df_results[col].sum()}")

                print("    Pour le Numéro Complémentaire:")
                for col in aw_cols_comp:
                    col_name_fr = col.replace('aw_comp_', '').replace('_', ' ').capitalize()
                    print(f"      {col_name_fr}: Moy. {df_results[col].mean():.2f}, Max {df_results[col].max()}, Total {df_results[col].sum()}")
    print("="*60)
    return df_backtest_custom, df_backtest_ml # Retourner les dataframes pour une analyse potentielle plus poussée


def predire_prochain_tirage_final(lotto_data_complet_desc, poids_optimises):
    print("\n" + "="*40 + "\n \"PRÉDICTION\" FINALE POUR LE PROCHAIN TIRAGE (HYPOTHÉTIQUE) \n" + "="*40)
    print("ATTENTION : Ceci est un exercice théorique basé sur des modèles statistiques et de ML. NE PAS UTILISER POUR DES PARIS RÉELS.")

    historique_complet_asc = lotto_data_complet_desc.sort_values(by='Date', ascending=True).reset_index(drop=True)
    if historique_complet_asc.empty: print("Aucune donnée historique pour la prédiction finale."); return

    print(f"Utilisation de {len(historique_complet_asc)} tirages historiques pour la prédiction finale.")

    stats_sur_historique_complet_custom = calculer_statistiques_avancees(historique_complet_asc.copy())
    nombre_total_tirages_historiques = len(historique_complet_asc)
    print(f"\n--- Prédiction avec Modèle Custom (utilisant les poids optimisés) ---")
    scores_p_custom_final = [{'numero':n, 'score':calculer_score_prediction_custom(n, stats_sur_historique_complet_custom, nombre_total_tirages_historiques, poids_optimises)} for n in range(1,50)]
    scores_c_custom_final = [{'numero':n, 'score':calculer_score_prediction_custom(f"C{n}", stats_sur_historique_complet_custom, nombre_total_tirages_historiques, poids_optimises)} for n in range(1,11)]
    scores_p_custom_final.sort(key=lambda x:x['score'], reverse=True)
    scores_c_custom_final.sort(key=lambda x:x['score'], reverse=True)
    pred_p_custom_final = sorted([item['numero'] for item in scores_p_custom_final[:5]])
    pred_c_custom_final = scores_c_custom_final[0]['numero'] if scores_c_custom_final and scores_c_custom_final[0]['score'] > 0 else None
    print(f"  Numéros principaux \"prédits\" (Custom): {pred_p_custom_final}")
    print(f"  Numéro complémentaire \"prédit\" (Custom): {pred_c_custom_final}")
    print("  Top 10 scores pour les numéros principaux (Custom):")
    for s in scores_p_custom_final[:10]: print(f"    N°{s['numero']}: score {s['score']:.3f}")


    if RUN_BACKTESTING_ML_IN_FULL_ANALYSIS:
        taille_fenetre_entrainement_final_ml = 105
        debut_fenetre_final = max(0, len(historique_complet_asc) - taille_fenetre_entrainement_final_ml)
        donnees_pour_entrainement_ml_final = historique_complet_asc.iloc[debut_fenetre_final:]
        min_hist_pour_features_ml_final = 21

        if len(donnees_pour_entrainement_ml_final) < min_hist_pour_features_ml_final:
            print(f"\nFenêtre d'entraînement ML final trop petite ({len(donnees_pour_entrainement_ml_final)}). Prédictions ML par défaut.")
            pred_p_ml_final = sorted(list(MES_NUMEROS_PRINCIPAUX_JOUES))[:5] if MES_NUMEROS_PRINCIPAUX_JOUES else random.sample(range(1,50),5)
            pred_c_ml_final = list(MES_COMPLEMENTAIRES_POSSIBLES_JOUES)[0] if MES_COMPLEMENTAIRES_POSSIBLES_JOUES else random.randint(1,10)
            predictions_probas_p_ml_final_display = []
        else:
            print(f"\n--- Prédiction avec Modèle ML (Random Forest) ---")
            print(f"  Entraînement des modèles ML finaux sur {len(donnees_pour_entrainement_ml_final)} tirages (indices {debut_fenetre_final} à {len(historique_complet_asc)-1}).")

            stats_pour_ml_final_prediction = calculer_statistiques_avancees(historique_complet_asc.copy())

            modeles_ml_p_final, modeles_ml_c_final = {}, {}
            predictions_probas_p_ml_final, predictions_probas_c_ml_final = [], []
            param_grid_rf_final = {'n_estimators': [50, 100], 'max_depth': [None, 10], 'min_samples_split': [2, 5], 'min_samples_leaf': [1, 3]}
            default_n_estimators_final = 100

            for num_p_ml in range(1, 50):
                X_train_ml, y_train_ml = preparer_donnees_pour_ml_numero(donnees_pour_entrainement_ml_final.copy(), num_p_ml, 'principal')
                if len(X_train_ml) < 10 or len(np.unique(y_train_ml)) < 2:
                    modeles_ml_p_final[num_p_ml] = None; predictions_probas_p_ml_final.append({'numero': num_p_ml, 'proba': 0.0}); continue

                model_ml_p_trained = None
                if USE_GRID_SEARCH_FOR_FULL_ANALYSIS_ML:
                    cv_splits_pf = min(2, len(X_train_ml) // (sum(param_grid_rf_final.get('min_samples_split', [2])) // len(param_grid_rf_final.get('min_samples_split', [2]))) if len(X_train_ml) > 10 else 2)
                    cv_splits_pf = max(2, cv_splits_pf)
                    grid_search_p_final = GridSearchCV(RandomForestClassifier(random_state=42, class_weight='balanced'), param_grid_rf_final, cv=cv_splits_pf, scoring='roc_auc', n_jobs=-1, error_score='raise')
                    try: grid_search_p_final.fit(X_train_ml, y_train_ml); model_ml_p_trained = grid_search_p_final.best_estimator_
                    except ValueError as e_gs_pf:
                        logging.warning(f"Erreur GridSearchCV Principal Final N°{num_p_ml}: {e_gs_pf}. Modèle par défaut.")
                        model_ml_p_trained = RandomForestClassifier(n_estimators=default_n_estimators_final, random_state=42, class_weight='balanced', n_jobs=-1).fit(X_train_ml, y_train_ml)
                else: model_ml_p_trained = RandomForestClassifier(n_estimators=default_n_estimators_final, random_state=42, class_weight='balanced', n_jobs=-1).fit(X_train_ml, y_train_ml)
                modeles_ml_p_final[num_p_ml] = model_ml_p_trained

                if num_p_ml not in stats_pour_ml_final_prediction:
                    logging.warning(f"Stats non trouvées N°P {num_p_ml} création features prédiction finale."); predictions_probas_p_ml_final.append({'numero': num_p_ml, 'proba': 0.0}); continue
                stat_num_cible_ml_p = stats_pour_ml_final_prediction[num_p_ml]
                s_t_p_pred_fin, a_3_d_pred_fin, a_5_d_pred_fin = _calculer_features_recence_pour_prediction(historique_complet_asc, num_p_ml, 'principal')
                ecart_moyen_spec_feature_pred_pf = stat_num_cible_ml_p.get('ecart_moyen_specifique', float('inf'))
                if ecart_moyen_spec_feature_pred_pf == float('inf'): ecart_moyen_spec_feature_pred_pf = len(historique_complet_asc) if stat_num_cible_ml_p.get('frequence_absolue',0) <=1 else -1
                features_pour_prediction_ml_p = np.array([[
                    stat_num_cible_ml_p.get('frequence_relative', 0.0), stat_num_cible_ml_p.get('ecart_actuel', len(historique_complet_asc)),
                    ecart_moyen_spec_feature_pred_pf, len(stat_num_cible_ml_p.get('ecarts_observes', [])),
                    s_t_p_pred_fin, a_3_d_pred_fin, a_5_d_pred_fin
                ]])
                if modeles_ml_p_final[num_p_ml]:
                    try: proba_p_ml = modeles_ml_p_final[num_p_ml].predict_proba(features_pour_prediction_ml_p)[0][1]; predictions_probas_p_ml_final.append({'numero': num_p_ml, 'proba': proba_p_ml})
                    except Exception as e_pred_pf: logging.error(f"Erreur predict_proba Principal Final N°{num_p_ml}: {e_pred_pf}"); predictions_probas_p_ml_final.append({'numero': num_p_ml, 'proba': 0.0})
                else: predictions_probas_p_ml_final.append({'numero': num_p_ml, 'proba': 0.0})

            for num_c_ml in range(1, 11):
                X_train_ml_c, y_train_ml_c = preparer_donnees_pour_ml_numero(donnees_pour_entrainement_ml_final.copy(), num_c_ml, 'complementaire')
                cle_stat_ml_c = f"C{num_c_ml}"
                if len(X_train_ml_c) < 10 or len(np.unique(y_train_ml_c)) < 2:
                    modeles_ml_c_final[num_c_ml] = None; predictions_probas_c_ml_final.append({'numero': num_c_ml, 'proba': 0.0}); continue

                model_ml_c_trained = None
                if USE_GRID_SEARCH_FOR_FULL_ANALYSIS_ML:
                    cv_splits_cf = min(2, len(X_train_ml_c) // (sum(param_grid_rf_final.get('min_samples_split', [2])) // len(param_grid_rf_final.get('min_samples_split', [2]))) if len(X_train_ml_c) > 10 else 2)
                    cv_splits_cf = max(2, cv_splits_cf)
                    grid_search_c_final = GridSearchCV(RandomForestClassifier(random_state=42, class_weight='balanced'), param_grid_rf_final, cv=cv_splits_cf, scoring='roc_auc', n_jobs=-1, error_score='raise')
                    try: grid_search_c_final.fit(X_train_ml_c, y_train_ml_c); model_ml_c_trained = grid_search_c_final.best_estimator_
                    except ValueError as e_gs_cf:
                        logging.warning(f"Erreur GridSearchCV Complémentaire Final N°{num_c_ml}: {e_gs_cf}. Modèle par défaut.")
                        model_ml_c_trained = RandomForestClassifier(n_estimators=default_n_estimators_final, random_state=42, class_weight='balanced', n_jobs=-1).fit(X_train_ml_c, y_train_ml_c)
                else: model_ml_c_trained = RandomForestClassifier(n_estimators=default_n_estimators_final, random_state=42, class_weight='balanced', n_jobs=-1).fit(X_train_ml_c, y_train_ml_c)
                modeles_ml_c_final[num_c_ml] = model_ml_c_trained

                if cle_stat_ml_c not in stats_pour_ml_final_prediction:
                    logging.warning(f"Stats non trouvées N°C {num_c_ml} création features prédiction finale."); predictions_probas_c_ml_final.append({'numero': num_c_ml, 'proba': 0.0}); continue
                stat_num_cible_ml_c = stats_pour_ml_final_prediction[cle_stat_ml_c]
                s_t_p_pred_fin_c, a_3_d_pred_fin_c, a_5_d_pred_fin_c = _calculer_features_recence_pour_prediction(historique_complet_asc, num_c_ml, 'complementaire')
                ecart_moyen_spec_feature_pred_cf = stat_num_cible_ml_c.get('ecart_moyen_specifique', float('inf'))
                if ecart_moyen_spec_feature_pred_cf == float('inf'): ecart_moyen_spec_feature_pred_cf = len(historique_complet_asc) if stat_num_cible_ml_c.get('frequence_absolue',0) <=1 else -1
                features_pour_prediction_ml_c = np.array([[
                    stat_num_cible_ml_c.get('frequence_relative', 0.0), stat_num_cible_ml_c.get('ecart_actuel', len(historique_complet_asc)),
                    ecart_moyen_spec_feature_pred_cf, len(stat_num_cible_ml_c.get('ecarts_observes', [])),
                    s_t_p_pred_fin_c, a_3_d_pred_fin_c, a_5_d_pred_fin_c
                ]])
                if modeles_ml_c_final[num_c_ml]:
                    try: proba_c_ml = modeles_ml_c_final[num_c_ml].predict_proba(features_pour_prediction_ml_c)[0][1]; predictions_probas_c_ml_final.append({'numero': num_c_ml, 'proba': proba_c_ml})
                    except Exception as e_pred_cf: logging.error(f"Erreur predict_proba Complémentaire Final N°{num_c_ml}: {e_pred_cf}"); predictions_probas_c_ml_final.append({'numero': num_c_ml, 'proba': 0.0})
                else: predictions_probas_c_ml_final.append({'numero': num_c_ml, 'proba': 0.0})

            predictions_probas_p_ml_final.sort(key=lambda x: x['proba'], reverse=True); predictions_probas_c_ml_final.sort(key=lambda x: x['proba'], reverse=True)
            pred_p_ml_final = sorted([item['numero'] for item in predictions_probas_p_ml_final[:5]])
            pred_c_ml_final = predictions_probas_c_ml_final[0]['numero'] if predictions_probas_c_ml_final and predictions_probas_c_ml_final[0]['proba'] > 0 else None
            predictions_probas_p_ml_final_display = predictions_probas_p_ml_final

        print(f"  Numéros principaux \"prédits\" (ML): {pred_p_ml_final}")
        print(f"  Numéro complémentaire \"prédit\" (ML): {pred_c_ml_final}")
        if predictions_probas_p_ml_final_display :
            print("  Top 10 probabilités pour les numéros principaux (ML):")
            for s in predictions_probas_p_ml_final_display[:10]: print(f"    N°{s['numero']}: proba {s['proba']:.3f}")
    else:
        print("\nPrédiction finale ML désactivée (RUN_BACKTESTING_ML_IN_FULL_ANALYSIS=False).")

    print("\nRAPPEL IMPORTANT : Les résultats de ce script sont purement théoriques et issus d'un exercice de modélisation.")
    print("Le Loto est un jeu de hasard. Ne basez aucune décision financière sur ces \"prédictions\".")
    print("="*60)

def analyser_performance_mes_numeros(lotto_data, numeros_principaux_joues, complementaires_joues_possibles):
    print(f"\n" + "="*40); print(f" ANALYSE DE PERFORMANCE POUR VOS NUMÉROS JOUÉS "); print(f" Numéros principaux joués: {numeros_principaux_joues}"); print(f" Complémentaires potentiels: {complementaires_joues_possibles}"); print(f"="*40)
    if lotto_data.empty:
        print("Aucune donnée pour analyse de vos numéros.");
        return None

    num_cols = [f"N{i}" for i in range(1, 6)]
    if not all(c in lotto_data.columns for c in num_cols) or 'Complementaire' not in lotto_data.columns:
        print("Colonnes manquantes.");
        return None

    # La variable 'nombre_total_tirages' est définie ici et sera utilisée plus bas.
    nombre_total_tirages = len(lotto_data)
    gains = {"5P":0,"5P+C":0,"4P":0,"4P+C":0,"3P":0,"3P+C":0,"2P":0,"2P+C":0,"1P+C":0,"0P+C":0}
    combinaison_jouee_gagnante_count, comp_6_gagnant, comp_7_gagnant, comp_9_gagnant = 0,0,0,0

    for _, tirage_row in lotto_data.iterrows():
        try:
            gagnants_p_set = {int(tirage_row[n]) for n in num_cols if pd.notna(tirage_row[n])}
            if len(gagnants_p_set) != 5 : logging.warning(f"Tirage incomplet: {tirage_row.get('Date','N/A')}"); continue
            comp_gagnant = int(tirage_row['Complementaire']) if pd.notna(tirage_row['Complementaire']) else None
            if comp_gagnant is None: logging.warning(f"Complémentaire manquant: {tirage_row.get('Date','N/A')}"); continue
        except ValueError: logging.error(f"Erreur conversion: {tirage_row.get('Date','N/A')}"); continue

        corrects_p = len(numeros_principaux_joues.intersection(gagnants_p_set))
        correct_c = comp_gagnant in complementaires_joues_possibles

        if correct_c:
            if comp_gagnant == 6: comp_6_gagnant +=1
            if comp_gagnant == 7: comp_7_gagnant +=1
            if comp_gagnant == 9: comp_9_gagnant +=1

        if corrects_p == 5 and correct_c: gains["5P+C"] += 1
        elif corrects_p == 5: gains["5P"] += 1
        elif corrects_p == 4 and correct_c: gains["4P+C"] += 1
        elif corrects_p == 4: gains["4P"] += 1
        elif corrects_p == 3 and correct_c: gains["3P+C"] += 1
        elif corrects_p == 3: gains["3P"] += 1
        elif corrects_p == 2 and correct_c: gains["2P+C"] += 1
        elif corrects_p == 2: gains["2P"] += 1
        elif corrects_p == 1 and correct_c: gains["1P+C"] += 1
        elif corrects_p == 0 and correct_c: gains["0P+C"] += 1

        if numeros_principaux_joues.issubset(gagnants_p_set): combinaison_jouee_gagnante_count +=1

    print(f"Sur {nombre_total_tirages} tirages analysés...")
    for rang, count in gains.items():
        if count > 0: print(f"  Rang '{rang}': {count} fois ({(count/nombre_total_tirages)*100:.2f}%)")

    print(f"\nCombinaison {numeros_principaux_joues} sortie en entier: {combinaison_jouee_gagnante_count} fois.")
    print(f"Complémentaire 6 (un de vos possibles) sorti: {comp_6_gagnant} fois.")
    print(f"Complémentaire 7 (un de vos possibles) sorti: {comp_7_gagnant} fois.")
    print(f"Complémentaire 9 (un de vos possibles) sorti: {comp_9_gagnant} fois.")

    if nombre_total_tirages > 0:
        k_joues = len(numeros_principaux_joues)
        k_corrects_cible = 3
        if k_joues >= k_corrects_cible:
            prob_kP_theorique = (nCr(k_joues, k_corrects_cible) * nCr(49-k_joues, 5-k_corrects_cible)) / nCr(49,5)
            attendus_kP = nombre_total_tirages * prob_kP_theorique
            print(f"\nPour '{k_corrects_cible}P' (vos {k_corrects_cible} principaux corrects parmi les 5 tirés, avec {k_joues} joués) :")
            print(f"  Observé : {gains.get(f'{k_corrects_cible}P',0)} fois")
            print(f"  Attendu (théoriquement) : {attendus_kP:.2f} fois (Proba: {prob_kP_theorique:.6f})")
    return gains

# ---- Exécution Principale ----
if __name__ == "__main__":
    start_time_script = time.time()
    os.makedirs("plots", exist_ok=True)
    print("Début du script Loto...")

    if RUN_LAMBDA_STRATEGY_MODE:
        print("\n" + "*"*10 + " MODE STRATÉGIE LAMBDA ACTIVÉ " + "*"*10)
        strategie_loto_lambda_main()
    else:
        print("\n" + "*"*10 + " MODE ANALYSE COMPLÈTE ACTIVÉ " + "*"*10)
        lotto_data_desc_full = fetch_lotto_data()

        if lotto_data_desc_full is not None and not lotto_data_desc_full.empty:
            print(f"\nDonnées totales récupérées initialement : {len(lotto_data_desc_full)} tirages.")

            print("Filtrage pour ne conserver que les tirages du Samedi (tous les mois)...")
            if 'Jour' in lotto_data_desc_full.columns and 'Date' in lotto_data_desc_full.columns:
                donnees_finales_filtrees_desc = lotto_data_desc_full[
                    lotto_data_desc_full['Jour'].astype(str).str.contains('Samedi', case=False, na=False)
                ].copy()
                if not donnees_finales_filtrees_desc.empty:
                    donnees_finales_filtrees_desc['Date'] = pd.to_datetime(donnees_finales_filtrees_desc['Date'], errors='coerce')
                    donnees_finales_filtrees_desc.dropna(subset=['Date'], inplace=True)
                print(f"Nombre de tirages du Samedi (tous mois) conservés : {len(donnees_finales_filtrees_desc)}.")
            else:
                print("Colonnes 'Jour' ou 'Date' manquantes. Utilisation des données non filtrées.")
                donnees_finales_filtrees_desc = lotto_data_desc_full.copy()


            if donnees_finales_filtrees_desc.empty:
                print("Aucun tirage après filtrage (ou aucune donnée initiale). Arrêt.")
            else:
                print(f"Nombre de tirages à analyser : {len(donnees_finales_filtrees_desc)}.")
                lotto_data_asc_filtrees = donnees_finales_filtrees_desc.sort_values(by='Date', ascending=True).reset_index(drop=True)

                if RUN_SLIDING_WINDOW_ENHANCED_METRICS:
                    analyze_sliding_windows(lotto_data_asc_filtrees.copy(), window_size=105, step_size=28)

                analysis_results = run_full_statistical_analysis(lotto_data_asc_filtrees.copy())
                analyser_performance_mes_numeros(lotto_data_asc_filtrees.copy(), MES_NUMEROS_PRINCIPAUX_JOUES, MES_COMPLEMENTAIRES_POSSIBLES_JOUES)

                if 'gaps' in analysis_results and analysis_results.get('gaps'):
                    gaps_data_result = analysis_results['gaps']
                    print(f"\n--- Analyse des Écarts pour VOS numéros joués ({MES_NUMEROS_PRINCIPAUX_JOUES}) ---")
                    for num_perso in sorted(list(MES_NUMEROS_PRINCIPAUX_JOUES)):
                        if num_perso in gaps_data_result and gaps_data_result.get(num_perso):
                            print(f"  Pour le numéro {num_perso}:")
                            print(f"    Écarts observés (partiel): {gaps_data_result[num_perso][:10]}...")
                            print(f"    Moyenne des écarts: {np.mean(gaps_data_result[num_perso]):.2f}")
                            print(f"    Écart max observé: {np.max(gaps_data_result[num_perso])}")
                            if num_perso in analysis_results.get('frequencies', {}):
                                derniere_apparition_index = -1
                                for idx, row_draw in reversed(list(lotto_data_asc_filtrees.iterrows())):
                                    try:
                                        numeros_du_tirage = {int(row_draw[f'N{k}']) for k in range(1,6) if pd.notna(row_draw[f'N{k}'])}
                                        if num_perso in numeros_du_tirage:
                                            derniere_apparition_index = idx; break
                                    except ValueError: logging.warning(f"Skipping row in gap (perso) due to int conversion: {row_draw}"); continue
                                if derniere_apparition_index != -1:
                                    ecart_actuel_perso = len(lotto_data_asc_filtrees) - 1 - derniere_apparition_index
                                    print(f"    Écart actuel (dataset filtré): {ecart_actuel_perso}")
                                else: print(f"    Écart actuel (dataset filtré): Jamais vu ou dernier tirage.")
                        else: print(f"  Pas de données d'écarts pour N°{num_perso} dans ce dataset.")


                min_hist_opti = 63
                min_tirages_backtest_opti = max(49, int(len(lotto_data_asc_filtrees) * 0.1))
                min_tirages_backtest_opti = min(min_tirages_backtest_opti, 105)

                if len(lotto_data_asc_filtrees) > min_hist_opti + min_tirages_backtest_opti:
                    poids_optimises_deterministe = optimiser_poids_par_grille(
                        lotto_data_asc_filtrees.copy(),
                        nombre_tirages_backtest=min_tirages_backtest_opti,
                        min_hist_req=min_hist_opti
                    )
                else:
                    print(f"\nPas assez de données ({len(lotto_data_asc_filtrees)}) pour l'optimisation des poids. Poids par défaut.")
                    poids_optimises_deterministe = {'frequence':0.1, 'proximite_ecart_moyen_spec':0.5, 'depassement_ecart_moyen_spec':0.3, 'proximite_ecart_global_ref':0.1, 'depassement_ecart_global_ref':0.05, 'bonus_jamais_sorti':0.1, 'bonus_rare':0.05}

                if RUN_TRANSITION_PROBABILITIES:
                    analyze_transitions(lotto_data_asc_filtrees.copy(), lag=1, top_n=15)

                if RUN_DRAW_CLUSTERING:
                    df_clustered = cluster_draws(lotto_data_asc_filtrees.copy(), CONFIG_LOTO_LAMBDA, n_clusters_kmeans=6)
                    if df_clustered is not None and 'cluster_kmeans' in df_clustered.columns:
                        print("\nAperçu des données avec clusters K-Means (si réussi):")
                        print(df_clustered[['Date', 'N1', 'N2', 'N3', 'N4', 'N5', 'cluster_kmeans']].tail().to_string())

                if RUN_PREFIXSPAN_ANALYSIS:
                    analyze_frequent_sequences_prefixspan(lotto_data_asc_filtrees.copy(), min_support_ratio=0.005, max_pattern_length=5)


                if RUN_BACKTESTING_ML_IN_FULL_ANALYSIS:
                    min_backtest_hist_global = 63
                    nb_backtest_tirages_a_tester = max(11, int(len(lotto_data_asc_filtrees) * 0.02))
                    nb_backtest_tirages_a_tester = min(nb_backtest_tirages_a_tester, 28) # Limiter le nombre de tirages pour le backtest
                    taille_fenetre_ml = 105

                    needed_for_ml_backtest = taille_fenetre_ml + nb_backtest_tirages_a_tester
                    needed_for_custom_backtest = min_backtest_hist_global + nb_backtest_tirages_a_tester

                    if len(lotto_data_asc_filtrees) >= max(needed_for_ml_backtest, needed_for_custom_backtest) :
                        # La fonction retourne maintenant les DataFrames pour une analyse plus poussée si besoin
                        df_results_custom, df_results_ml = tester_performance_predictive_historique(
                            lotto_data_asc_filtrees.copy(),
                            nombre_de_tirages_a_tester=nb_backtest_tirages_a_tester,
                            poids_optimises=poids_optimises_deterministe
                        )
                        # Vous pouvez maintenant faire quelque chose avec df_results_custom et df_results_ml
                        # Par exemple, sauvegarder en CSV ou afficher plus de stats
                        if df_results_custom is not None and not df_results_custom.empty: # Vérification ajoutée
                             print("\nDataFrame des résultats détaillés du backtest (Modèle Custom):")
                             print(df_results_custom.head().to_string())
                        if df_results_ml is not None and not df_results_ml.empty: # Vérification ajoutée
                             print("\nDataFrame des résultats détaillés du backtest (Modèle ML):")
                             print(df_results_ml.head().to_string())

                    else: print(f"\nPas assez de données pour backtesting comparatif.")
                else: print("\nBacktesting ML désactivé (RUN_BACKTESTING_ML_IN_FULL_ANALYSIS=False).")

                if len(lotto_data_asc_filtrees) > min_hist_opti : # Utiliser min_hist_opti qui est la même que pour l'optimisation
                    predire_prochain_tirage_final(donnees_finales_filtrees_desc.copy(), poids_optimises_deterministe)
                else: print(f"\nPas assez de données pour la prédiction finale.")
        else:
            print("Aucune donnée de loterie n'a pu être chargée initialement. Arrêt du script.")

    end_time_script = time.time()
    print(f"\nScript terminé. Durée totale d'exécution : {time.strftime('%H:%M:%S', time.gmtime(end_time_script - start_time_script))}.")

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject